# Twitter Sentiment Analysis of Public Reaction to COVID-19 News

**Project Overview:**

This project aims to analyze a large dataset of COVID-19-related tweets to understand how public sentiment evolves and spreads in response to news announcements and events. By leveraging natural language processing (NLP) techniques and sentiment analysis models, we seek to gain valuable insights into the dynamics of online conversations surrounding the pandemic.

**Importance and Motivation:**

Understanding public sentiment during a global crisis like the COVID-19 pandemic is crucial for various stakeholders, including:

- **Public Health Officials:** To gauge public response to health policies and interventions.
- **Media Outlets:** To assess the impact of their news coverage on public perception.
- **Government Agencies:** To monitor public opinion and tailor communication strategies.
- **Researchers:** To study the spread of information and misinformation on social media.

This project contributes to this understanding by providing a comprehensive analysis of Twitter data, revealing trends and patterns in public sentiment related to COVID-19.

Notebook Structure
---
<details>
<summary><b>1. Business Problem and Objectives</b></summary>
   Define the problem being addressed and its relevance to real-world scenarios.
</details>

---

<details>
<summary><b>2. Data Acquisition and Preparation</b></summary>

- ### **2.1 Data Source and Download**  
  Explanation of the dataset source and how it was obtained.  

- ### **2.2 Installing Required Modules**  
  List and install the libraries needed for the project.  

- ### **2.3 Importing Modules and Global Variables**  
  Set up imports and define constants or global variables.  

- ### **2.4 Defining Supplemental Functions**  
  Helper functions to streamline data processing.  

- ### **2.5 Data Loading**  
  Load the dataset into a DataFrame or suitable data structure.  

- ### **2.6 Basic Data Understanding**  
  Perform initial data exploration, including shape, columns, and types.  

- ### **2.7 Data Cleaning and EDA: Date**  
  Extract and analyze temporal trends in the dataset.

- ### **2.8 Data Cleaning and EDA: Language**  
  Extract relevant language subset.

- ### **2.9 Data Cleaning and EDA: Location**  
  Process location data to standardize and extract insights.  

- ### **2.10 Data Cleaning and EDA: Source**  
  Analyze the platforms from which tweets were sent.  

- ### **2.11 Data Cleaning and EDA: Sentiment**  
  Explore sentiment labels and their distribution.  

- ### **2.12 Exploratory Data Analysis (EDA): Social Connections**  
  Investigate user mentions, retweets, and network connections.
</details>

---

<details>
<summary><b>3. Text Preprocessing and Feature Engineering</b></summary>

- ### **3.1 Cleaning**  
  Remove noise, including special characters, links, mentions, and hashtags.  

- ### **3.2 Preprocessing**  
  Tokenize, lemmatize, and remove stop words from the text data.  

- ### **3.3 Feature Extraction**  
  Generate n-grams, TF-IDF features, or embeddings for model input.  

</details>

---

<details>
<summary><b>4. Sentiment Analysis</b></summary>

- ### **4.1 Train-Test Data Split**  
- ### **4.2 Base Model Training, Evaluation**   
  Choose a simple base classification model and train it on the preprocessed data. Assess model performance using metrics like accuracy, precision, and recall.
- ### **4.3 Model Hyperparameter Search**
  Vary base model parameters to see if it improves model's perfomance.      
- ### **4.4 Insights and Visualization**  
  Visualize results and discuss findings, including strengths and limitations.
- ### **4.5 Other Models Tested**
  List other models tested, their parameters and performance.   
</details>

---

<details>
<summary><b>6. Conclusion</b></summary>


</details>




## 1.1 Business Problem and Objectives

**Problem Statement:**

Media outlets and public health organizations need a better understanding of how their COVID-19-related news and announcements influence public sentiment on Twitter. This project addresses this need by analyzing a large dataset of tweets to identify and track sentiment trends in response to news events.

**Key Questions:**

- How do positive and negative sentiments spread among users following a COVID-19 news announcement?
- What are the key topics and themes associated with different sentiment trends?
- Can we identify any patterns or correlations between news events and changes in public sentiment?

**Project Objectives:**

- To develop a robust NLP pipeline for cleaning, preprocessing, and analyzing Twitter data.
- To apply sentiment analysis models to classify tweets and track sentiment trends over time.
- To visualize and interpret the sentiment analysis results to provide actionable insights.
- To potentially identify key influencers and networks driving sentiment on Twitter.


# 2. Data Acquisition and Preparation


## 2.1. Data Understanding.


This section outlines the source of the data used in this project,  and provides instructions for downloading it.

**Data Sources:**

1. **Covid-19 Twitter Dataset:** The primary dataset for this Twitter sentiment analysis project is the "Covid-19 Twitter Dataset" available on Kaggle. This dataset contains a large collection of tweets related to COVID-19, including tweet text, user details, location, and sentiment labels.

2. **GloVe Embeddings:** To enhance the analysis, we will utilize pre-trained GloVe embeddings from Stanford NLP. These word embeddings capture semantic relationships between words and can improve the performance of NLP models.

**Data Relevance**:

* The Covid-19 Twitter Dataset contains a vast collection of tweets related to the pandemic, providing a valuable source of public opinion and sentiment during this period.
* This dataset is suitable for our project because it includes sentiment labels, allowing us to train and evaluate sentiment analysis models.


**Data Limitations**

* Recent changes to the Twitter API have significantly impacted the accessibility of tweet data for research and analysis.  Specifically, Twitter has severely restricted free API access. This means that retrieving the original dataset used in this project [(see Panacea's lab github page)](https://github.com/thepanacealab/covid19_twitter) is no longer possible without incurring substantial costs.
* This dataset contains pre-computed sentiment labels, however, we don't know what method was used and how the accuracy of the sentiment was evaluated.

**Download Instructions:**

1. **Kaggle Dataset:** The dataset can be accessed and downloaded from the following Kaggle page:
    [Covid-19 Twitter Dataset](https://www.kaggle.com/datasets/arunavakrchakraborty/covid19-twitter-dataset/data)

2. ** GloVe Embeddings:** The pre-trained GloVe embeddings can be obtained from the [Stanford NLP website](https://nlp.stanford.edu/projects/glove/). For this project, we will use the "glove.twitter.27B.zip" file, which contains 10-200-dimensional embeddings trained on 27B Twitter tokens.

**Data Storage:**

   - **Local Execution:** If you are running the notebook locally, please download the dataset files and place them into a folder named `Data` within your project directory.
  
   - **Colab Environment:** If you are using Google Colab, it's best recommended to mount a google drive.

**Data Loading:**

1. **Loading Datasets:** code would check runtime environment and for local environment set the `data_dir` to be `Data` folder on the same level as the notebook. If you're using Colab, you'll need to adjust the defauld directory. Highly recommend to not use runtime-dependent directory.   
2. **Loading GloVe Embeddings:** The script will automatically check if the GloVe embeddings are already present in the 'Data' directory. If not, it will download and extract them for you:
```
download_and_extract_glove(data_dir)
```

## 2.2 Installing Required Modules

This section focuses on installing the necessary Python libraries and packages required for our Twitter sentiment analysis project. We accomplish this through the following steps:

1. **Requirements File:**
    - We retrieve the list of required packages from a `requirements.txt` file hosted on GitHub using `wget`. This file contains the names and versions of all the dependencies.
    - This ensures that we install the correct versions of the libraries for compatibility and reproducibility.
    - Here's the link to the requirements file on GitHub:
       `https://raw.githubusercontent.com/leksea/capstone-twitter-sentiment-analysis/main/requirements.txt`

2. **Installation using pip:**
    - We use Python's `pip` package manager to install the libraries listed in the `requirements.txt` file.
    - The `-r` flag instructs `pip` to read the requirements file and install all the packages listed within.

In [1]:
!wget https://raw.githubusercontent.com/leksea/capstone-twitter-sentiment-analysis/main/requirements.txt
!pip install -r 'requirements.txt'

--2025-01-02 05:46:42--  https://raw.githubusercontent.com/leksea/capstone-twitter-sentiment-analysis/main/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 165 [text/plain]
Saving to: ‘requirements.txt.1’

requirements.txt.1  100%[===================>]     165  --.-KB/s    in 0s      

2025-01-02 05:46:43 (2.92 MB/s) - ‘requirements.txt.1’ saved [165/165]



## 2.3 Importing Modules and Global Variables

This section focuses on setting up the necessary environment for our analysis by importing the required Python modules and declaring global variables. We perform the following:

1. **Module Imports:** We import a variety of modules that will be essential for data manipulation, analysis, visualization, and natural language processing tasks. These modules include:

    - **Built-in Modules:** `os`, `string`, `re`, `glob`, `time`, `datetime`, `concurrent.futures`, `json`, `collections`, `concurrent`.
    - **Data Processing and Analysis:** `numpy`, `pandas`.
    - **Visualization:** `matplotlib.pyplot`, `seaborn`, `networkx`, `folium`, `branca.colormap`, `cartopy`.
    - **Natural Language Processing (NLP):** `nltk`, `emot`, `emoji`.
    - **Machine Learning:** `sklearn`.

2. **Global Variable Declarations:**
    - We define and initialize global variables that will be used throughout the analysis. These include:
        - `geolocator`: An instance of the `Nominatim` geolocator from the `geopy` library for location standardization.
        - `tqdm`: Enabling progress bars for long computations using `tqdm.pandas()`.
        - `stop_words`: A set of English stop words from `nltk.corpus` for text preprocessing.
        - `lemmatizer`: An instance of the `WordNetLemmatizer` from `nltk.stem` for lemmatization.

3. **Downloading NLP Resources:**
    - We download necessary resources for NLP tasks, such as stopwords, wordnet, and punkt using `nltk.download()`.

In [2]:
# built-in modules
import os
import string
import re
import glob
import time
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed
import json
from collections import Counter
import os
import zipfile
from pathlib import Path
# Optimization with Parallel Computing:
from concurrent.futures import ProcessPoolExecutor
# url processing for extracting the coordinates
import requests
# progress bar monitoring
from tqdm import tqdm
# data manupulation, analysis, sparce matrix
import numpy as np
import pandas as pd
from numpy.random import rand, randint
from scipy.sparse import hstack, csr_matrix
# general data visualization
import matplotlib.pyplot as plt
import seaborn as sns
# networkx for user connection visualization
import networkx as nx
# world maps
import folium
from folium import plugins
from folium.plugins import HeatMap
import branca.colormap as cm
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
# displaying the folium heatmap
from IPython.display import display, HTML
#for location standartization
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
# world cloud
from wordcloud import WordCloud
# Natural Language Processing (NLP)
import nltk
from emot.emo_unicode import UNICODE_EMOJI
import emoji
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk import pos_tag, ne_chunk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
# Embeddings
from transformers import BertTokenizer, BertModel
import torch
# Classification
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
%matplotlib inline
## GLOBAL VARIABLES
# stop words for tokenizer
nltk.download('stopwords')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')
# Initialize geolocator globally for efficient geocoding cache
geolocator = Nominatim(user_agent="batch-geocoding")
# Enable tqdm for pandas, progress bar for long computations
tqdm.pandas()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


## 2.4 Defining Supplemental Functions


* ```def determine_data_dir()```
* ```def download_and_extract_glove(data_dir)```
* ```def display_categorical_vals(df)```
* ```def has_special_chars(location) ```
* ```def save_cache_to_json(cache, file_path="location_cache.json")```
* ```def load_cache_to_json(cache, file_path="location_cache.json")```
* ```def geocode_location(location)```
* ```def batch_geocode(locations)```
* ```def extract_word(location, position="first")```
* ```def split_geocoded_location(location)```
* ```def get_coordinates(input_type, name, output_as='center', retries=3, delay=5)```
* ```def add_coordinates_with_progress(df, city_col='city', state_col='state', country_col='country')```
* ```def color(magnitude)```
* ```def generateBaseMap(input_type, df, default_location=[40.693943, -73.985880], default_zoom_start=2)```
* ```def extract_html_source(source_text)```
* ```def replace_emoticons_with_emojis(text)```
* ```def process_tweet_data(tweet, emoji_list=None)```
* ``` def clean_tweets_with_progress_parallel(df, text_col='original_text', num_processes=6)```
* ```def preprocess_text(df, text_column)```
* ```def compute_ngrams(df, text_column, ngram_range=(2, 3), max_features=5000)```
* ```def bert_embedding(sentence, tokenizer, model)```
* ```def plot_normalized_confusion_matrix(y_true, y_pred, class_labels, title="Confusion Matrix", cmap="Blues")```

 **Optional**

* ``` def fitness_function(selected_features, X, y, model, cv=3)```
* ```def update_gwo(population, alpha, beta, delta,a)```
* ```def hybrid_gwo_abc(X, y, model, n_wolves=10, n_iter=20)```



In [3]:
# Supplemental function to determine data directory
# Input: none
# Output: Data directory, depending on runtime environment.

def determine_data_dir():
    """
    Determines the data directory based on the execution environment:
    - Local: Uses 'Data' directory in the current working directory.
    - Cloud (e.g., Google Colab): Uses '/content' as the data directory.

    Returns:
        str: Path to the appropriate data directory.
    """
    if 'COLAB_GPU' in os.environ:  # Check if running in Google Colab
        data_dir = "/content/drive/MyDrive/Colab_Notebooks/Data"
        print(f"Running in Google Colab. Using data directory: {data_dir}")
    else:
        data_dir = os.path.join(os.getcwd(), "Data")
        print(f"Running locally. Using data directory: {data_dir}")

        # Ensure the 'Data' directory exists locally
        if not os.path.isdir(data_dir):
            print(f"The directory '{data_dir}' does not exist. Please create it and place the data files there.")
            raise FileNotFoundError(f"'{data_dir}' directory is required for local execution.")

    return data_dir

In [4]:
def download_and_extract_glove(data_dir):
    """
    Downloads GloVe embeddings from Stanford, extracts them, and copies the specified file to data_dir.
    """
    # Check if the file is already present in the data_dir, only download if it's not there
    glove_file = Path(data_dir) / "glove.twitter.27B.200d.txt"
    if not glove_file.is_file():
        print("Downloading GloVe embeddings...")
        # Define GloVe URLs
        glove_zip_url = "http://nlp.stanford.edu/data/glove.twitter.27B.zip"
        zip_file_path = os.path.join(data_dir, "glove.twitter.27B.zip")

        # Download GloVe
        os.system(f"wget {glove_zip_url} -O {zip_file_path}")

        # Extract GloVe
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(data_dir)

        # Remove the downloaded ZIP file
        os.remove(zip_file_path)

        print(f"GloVe embeddings downloaded and extracted to {data_dir}")
    else:
        print("GloVe embeddings are already present in the data directory.")

In [5]:
# Supplemental function will display unique values for all categorical columns in a dataframe.
def display_categorical_vals(df):
    # select categorical columns
    categorical_columns = df.select_dtypes(include=['object', 'category']).columns

    # print categorical columns and their unique values
    for col in categorical_columns:
        unique_values = df[col].unique()
        print(f"Column '{col}' has unique values: {unique_values}")

In [6]:
# define a function to check for special characters
def has_special_chars(location):
    return bool(re.search(r'[^\w\s,.-]', location))  # check for non-alphanumeric and non-space chars

In [7]:
def save_cache_to_json(cache, file_path="location_cache.json"):
    """
    Saves the location cache to a JSON file.

    Args:
        cache (dict): The cache dictionary to save.
        file_path (str): The file path where the cache will be saved.
    """
    with open(file_path, "w") as f:
        json.dump(cache, f)
    print(f"Cache saved to {file_path}")

In [8]:
def load_cache_from_json(file_path="location_cache.json"):
    """
    Loads the location cache from a JSON file.

    Args:
        file_path (str): The file path from where the cache will be loaded.

    Returns:
        dict: The loaded cache dictionary.
    """
    try:
        with open(file_path, "r") as f:
            cache = json.load(f)
        print(f"Cache loaded from {file_path}")
        return cache
    except FileNotFoundError:
        print(f"No cache file found at {file_path}. Starting with an empty cache.")
        return {}

In [9]:
# Dictionary to cache geocoding results
try:
    # Initialize cache from file
    data_dir = determine_data_dir()
    location_cache_file = os.path.join(data_dir, "location_cache.json")  # Path to cache file
    location_cache = load_cache_from_json(location_cache_file)
except FileNotFoundError:
    # If the file doesn't exist, initialize an empty cache
    location_cache = {}

# Supplemental function to use a geocoding API for location resolution
def geocode_location(location):
    """
    Resolve location using a geocoding API with caching.
    Returns results in City, State, Country format.
    """
    # Check cache first
    if location in location_cache:
        return location_cache[location]

    try:
        # Add a delay to respect API rate limits
        geo = geolocator.geocode(location, addressdetails=True, exactly_one=True, timeout=10)
        if geo:
            # Default: Extract the address components
            address = geo.raw.get('address', {})
            city = address.get('city') or address.get('town') or address.get('village') or address.get('hamlet')
            state = address.get('state')
            country = address.get('country')

            # Fallback: Parse city and country from display_name if missing
            if not city:
                try:
                    city = geo.raw['display_name'].split(',')[0].strip()
                except (KeyError, IndexError):
                    city = "Unknown"
            if not country:
                try:
                    country = geo.raw['display_name'].split(',')[-1].strip()
                except (KeyError, IndexError):
                    country = "Unknown"
            # Fallback: Parse state dynamically from display_name if missing or ambiguous
            if not state:
                try:
                    components = geo.raw['display_name'].split(',')
                    components = [comp.strip() for comp in components]
                    for i in range(len(components) - 1, -1, -1):  # Iterate backwards
                        if 'County' not in components[i] and "United States" not in components[i]:
                            state = components[i]
                            break
                except (KeyError, IndexError):
                    state = "Unknown"

            # Avoid redundancy: "Country, Unknown, Country"
            if city == country:
                city = "Unknown"
            if state == country:
                state = "Unknown"

            # Construct the result in the desired format
            result = f"{city}, {state}, {country}"
        else:
            result = "Unknown"
    except (GeocoderTimedOut, GeocoderServiceError) as e:
        result = f"Error: {e}"

    # Cache the result
    location_cache[location] = result
    return result

Running in Google Colab. Using data directory: /content/drive/MyDrive/Colab_Notebooks/Data
Cache loaded from /content/drive/MyDrive/Colab_Notebooks/Data/location_cache.json


In [10]:
# geocode multiple locations in parallel using ThreadPoolExecutor
def batch_geocode(locations):
    """
    Geocode multiple locations in parallel using ThreadPoolExecutor.
    """
    results = []
    with ThreadPoolExecutor(max_workers=5) as executor:  # max_workers can be adjusted as needed
        # Use tqdm to wrap the executor's map method for progress tracking
        for result in tqdm(executor.map(geocode_location, locations), \
                           total=len(locations), desc="Geocoding Progress"):
            results.append(result)
    return results

In [11]:
# function to extract the first or last word from a location string
def extract_word(location, position="first"):
    """
    Extract the first or last word from a location string.

    Args:
        location (str): The location string to process.
        position (str): 'first' to extract the first word, 'last' to extract the last word.

    Returns:
        str: The extracted word or 'Unknown' if the location is empty or invalid.
    """
    words = location.split()
    if words:
        return words[0] if position == "first" else words[-1]
    return "Unknown"

In [12]:
# function to split geocoded_location into City, State, and Country
def split_geocoded_location(location):
    if pd.notna(location):
        parts = location.split(",")
        parts = [p.strip() for p in parts]  # remove extra whitespace
        city = parts[0] if len(parts) > 0 else "Unknown"
        state = parts[1] if len(parts) > 1 else "Unknown"
        country = parts[2] if len(parts) > 2 else "Unknown"
        return city, state, country
    return "Unknown", "Unknown", "Unknown"

In [13]:
# Dictionary to cache coordinates
try:
    # Initialize cache from file
    data_dir = determine_data_dir()
    coordinate_cache_file = os.path.join(data_dir, "coordinate_cache.json")  # Path to cache file
    coordinate_cache = load_cache_from_json(coordinate_cache_file)
except FileNotFoundError:
    # If the file doesn't exist, initialize an empty cache
    coordinate_cache = {}

# Update get_coordinates function to include caching
def get_coordinates(input_type, name, output_as='center', retries=3, delay=5):
    """
    Fetch coordinates of a city/state/country using Nominatim API with caching and retry logic.

    Args:
        input_type (str): 'country', 'state', or 'city' to specify the type of input.
        name (str): Name of the location.
        output_as (str): 'center' or 'boundingbox' for coordinate type.
        retries (int): Number of retry attempts.
        delay (int): Delay between retries in seconds.

    Returns:
        list: [latitude, longitude]. Returns [0, 0] on failure.
    """
    # Check the cache first
    if name in coordinate_cache:
        return coordinate_cache[name]

    url = f"http://nominatim.openstreetmap.org/search"
    headers = {
        "User-Agent": "ColabGeocoder/1.0 (leksea@gmail.com)"
    }
    params = {
        input_type: name,
        "format": "json",
        "polygon": 0
    }

    for attempt in range(retries):
        try:
            response = requests.get(url, headers=headers, params=params)
            response.raise_for_status()
            data = response.json()
            if data:
                if output_as == 'center':
                    result = [float(data[0]['lat']), float(data[0]['lon'])]
                elif output_as == 'boundingbox':
                    result = [float(coord) for coord in data[0]['boundingbox']]
                else:
                    result = [0, 0]
                # Cache the result
                coordinate_cache[name] = result
                return result
        except requests.exceptions.RequestException as e:
            print(f"Error fetching coordinates for {name}: {e}")
            if attempt < retries - 1:
                print(f"Retrying in {delay} seconds... ({attempt + 1}/{retries})")
                time.sleep(delay)
            else:
                print(f"Failed to fetch coordinates for {name} after {retries} attempts.")
                return [0, 0]

    # Cache failed attempt as [0, 0] to avoid repeated retries
    coordinate_cache[name] = [0, 0]
    return [0, 0]

Running in Google Colab. Using data directory: /content/drive/MyDrive/Colab_Notebooks/Data
Cache loaded from /content/drive/MyDrive/Colab_Notebooks/Data/coordinate_cache.json


In [14]:
# add_coordinates_with_progress function
def add_coordinates_with_progress(df, city_col='city', state_col='state', country_col='country'):
    """
    Add latitude and longitude coordinates to a DataFrame based on unique combinations
    of City, State, and Country, only for rows where these are not 'Unknown'.

    Args:
        df (pd.DataFrame): The input DataFrame.
        city_col (str): Column name for city.
        state_col (str): Column name for state.
        country_col (str): Column name for country.

    Returns:
        pd.DataFrame: The updated DataFrame with 'Latitude' and 'Longitude' columns.
    """
    # Create a unique DataFrame of City, State, Country combinations
    unique_locations = df[(df[city_col] != 'Unknown') &
                          (df[state_col] != 'Unknown') &
                          (df[country_col] != 'Unknown')][[city_col, state_col, country_col]].drop_duplicates()

    # Define a helper function to fetch coordinates
    def fetch_coords(row):
        location_name = f"{row[city_col]}, {row[state_col]}, {row[country_col]}"
        return get_coordinates('city', location_name)

    # Add Latitude and Longitude columns to the unique locations
    unique_locations[['latitude', 'longitude']] = unique_locations.progress_apply(fetch_coords, axis=1, result_type='expand')

    # Create a mapping dictionary for efficient lookup
    location_to_coords = unique_locations.set_index([city_col, state_col, country_col])[['latitude', 'longitude']].to_dict('index')

    # Initialize Latitude and Longitude in the main DataFrame
    df['latitude'], df['longitude'] = 0.0, 0.0

    # Map coordinates back to the original DataFrame
    for index, row in df.iterrows():
        key = (row[city_col], row[state_col], row[country_col])
        if key in location_to_coords:
            df.at[index, 'latitude'] = location_to_coords[key]['latitude']
            df.at[index, 'longitude'] = location_to_coords[key]['longitude']

    return df

In [15]:
# Function to determine marker color based on tweet count
def color(magnitude):
    """
    Returns a color based on the magnitude using a hot-to-cool color map.
    """
    if magnitude >= 2000:
        return 'red'  # Hot color for high magnitude
    elif 500 <= magnitude < 2000:
        return 'orange'  # Medium-hot color
    elif 100 <= magnitude < 500:
        return 'yellow'  # Neutral color
    elif 50 <= magnitude < 100:
        return 'lightblue'  # Medium-cool color
    else:
        return 'blue'  # Cool color for low magnitude

# Function to generate the heatmap
def generateBaseMap(input_type, df, default_location=[37.774929, -122.419416], default_zoom_start=2):
    """
    Function to generate a heatmap with markers for tweet distribution.

    Args:
        input_type (str): 'country' or 'city' to specify the type of heatmap.
        df (pd.DataFrame): DataFrame containing latitude, longitude, tweet count, and name.
        default_location (list): Default map center location as [latitude, longitude].
        default_zoom_start (int): Default zoom level for the map.

    Returns:
        folium.Map: A folium map object with heatmap and markers.
    """
    # Initialize the base map
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    marker_cluster = plugins.MarkerCluster().add_to(base_map)

    # Add the heatmap
    HeatMap(data=df[['latitude', 'longitude']].values.tolist(), radius=20, max_zoom=13).add_to(base_map)

    # Add markers with popups
    for lat, lon, tweet_count, name in zip(df['latitude'], df['longitude'], df['tweet_count'], df.iloc[:, 0]):
        popup_content = folium.Popup(f"{name}<br>{tweet_count} tweets", max_width=300)
        folium.Marker(
            location=[lat, lon],
            popup=popup_content,
            icon=folium.Icon(color=color(tweet_count), icon='twitter', prefix='fa')
        ).add_to(marker_cluster)

    # Add a colormap legend
    min_val, max_val = df['tweet_count'].min(), df['tweet_count'].max()
    colormap = cm.LinearColormap(colors=['blue', 'yellow', 'red'], vmin=min_val, vmax=max_val)
    colormap.caption = f"{input_type.title()} Distribution of COVID-19 Tweets"
    colormap.add_to(base_map)

    return base_map

In [16]:
# function to extract readable source from HTML content
def extract_html_source(source_text):
    """
    Extracts the readable text (e.g., 'Twitter for Android') from the source HTML string.

    Args:
        source_text (str): The raw HTML string in the source column.

    Returns:
        str: The cleaned, readable source text.
    """
    return re.sub(r'<.*?>', '', str(source_text)).strip()  # remove HTML tags

In [17]:
#define a dictionary mapping emoticons to emojis
emoticon_to_emoji = {
    ":)": "😊",
    ":D": "😃",
    ":(": "☹️",
    ":/": "😕",
    ":P": "😛",
    ";)": "😉",
    ":'(": "😢",
    ":o": "😮",
    ":|": "😐",
    ":))": "😂",
    ":*": "😘",
    "xD": "😆"
}

def replace_emoticons_with_emojis(text):
    """
    Replaces emoticons in the text with corresponding emojis.

    Args:
        text (str): The input text.

    Returns:
        str: Text with emoticons replaced by emojis.
    """
    # Use regex to find and replace emoticons
    for emoticon, emoji in emoticon_to_emoji.items():
        text = re.sub(re.escape(emoticon), emoji, text)
    return text

In [18]:
# big cleaning function
def process_tweet_data(tweet, emoji_list=None):
    """
    Processes a tweet to extract mentions, hashtags, retweets, emojis, hyperlinks, and cleaned text.

    Args:
        tweet (str): The raw tweet text.
        emoji_list (list): List of emojis to extract. Defaults to keys of UNICODE_EMOJI.

    Returns:
        dict: Dictionary with extracted components and cleaned text.
    """
    # ensure input is a string
    tweet = str(tweet)
    # default emoji list if not provided
    if emoji_list is None:
        emoji_list = list(UNICODE_EMOJI.keys())

    # extract mentions
    mentions = re.findall(r'@\w+', tweet)

    # extract hashtags
    hashtags = re.findall(r'#\w+', tweet)

    # check for retweets and extract username after RT
    retweets = re.findall(r'^RT @(\w+)', tweet)
    retweet_user = retweets[0] if retweets else None

    # extract hyperlinks before emoji
    hyperlinks = re.findall(r'https?://[^\s]+|www\.[^\s]+', tweet)
    tweet = re.sub(r'https?://[^\s]+|www\.[^\s]+', '', tweet) #Remove URL

    # extract emojis
    tweet = replace_emoticons_with_emojis(tweet)
    emojis = ''.join([char for char in tweet if char in emoji_list])

    # replace emojis with text
    tweet = emoji.demojize(tweet).replace('_', ' ')

    # remove mentions, hashtags, retweets, emojis, and hyperlinks from the tweet
    cleaned_text = re.sub(r'@\w+', '', tweet)  # Remove mentions
    cleaned_text = re.sub(r'#\w+', '', cleaned_text)  # Remove hashtags
    cleaned_text = re.sub(r'^RT', '', cleaned_text)   # Remove retweets
    cleaned_text = ''.join([char for char in cleaned_text if char not in emoji_list])  # Remove emojis

    # remove special characters, extra spaces, numbers
    cleaned_text = re.sub(r'[{}]'.format(re.escape(string.punctuation)), '', cleaned_text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    cleaned_text = re.sub(r'\d+', '', cleaned_text)

    #lowercase the text, remove numbers
    cleaned_text = cleaned_text.lower()
    return {
        'mentions': mentions,
        'hashtags': hashtags,
        'retweets': retweets,
        'emojis': emojis,
        'hyperlinks': hyperlinks,
        'cleaned_text': cleaned_text
    }

In [19]:
def extract_cleaned_text(tweet):
    """Helper function to extract 'cleaned_text' from process_tweet_data."""
    return process_tweet_data(tweet)['cleaned_text']

def clean_tweets_with_progress_parallel(df, text_col='original_text', num_processes=6):
    """
    Cleans tweet data in parallel using ProcessPoolExecutor.

    Args:
        df (pd.DataFrame): DataFrame containing tweets.
        text_col (str): Column name containing the tweet text.
        num_processes (int): Number of processes to use for parallel execution.

    Returns:
        pd.DataFrame: Updated DataFrame with new columns for cleaned text and extracted components.
    """
    with ProcessPoolExecutor(max_workers=num_processes) as executor:
         # Extract only the 'cleaned_text' from process_tweet_data using the helper function
        cleaned_texts = list(tqdm(executor.map(extract_cleaned_text, df[text_col]),
                                   total=len(df), desc="Cleaning Tweets"))

    # Return as a pandas Series
    return pd.Series(cleaned_texts, index=df.index, name='cleaned_text')

In [20]:
# Stop words and lemmatizer initialization
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(df, text_column):
    """
    Preprocesses text data by tokenizing, removing stop words, and lemmatizing.

    Args:
        df (pd.DataFrame): Input DataFrame containing the text column.
        text_column (str): Name of the column with raw text to preprocess.

    Returns:
        pd.DataFrame: DataFrame with processed text columns.
    """
    # Convert 'text_column' to string type to ensure it contains only string values
    df[text_column] = df[text_column].astype(str)
    # Tokenization
    df['tokenized_text'] = df[text_column].progress_apply(word_tokenize)

    # Stop word removal
    df['filtered_tokens'] = df['tokenized_text'].progress_apply(
        lambda tokens: [word for word in tokens if word.lower() not in stop_words]
    )

    # Lemmatization
    df['lemmatized_tokens'] = df['filtered_tokens'].progress_apply(
        lambda tokens: [lemmatizer.lemmatize(word) for word in tokens]
    )

    # Combine tokens back into text for n-gram computation
    df['lemmatized_text'] = df['lemmatized_tokens'].progress_apply(lambda tokens: ' '.join(tokens))

    return df

In [21]:
def compute_ngrams(df, text_column, ngram_range=(2, 3), max_features=5000):
    """
    Computes n-grams from preprocessed text data.

    Args:
        df (pd.DataFrame): Input DataFrame containing the processed text column.
        text_column (str): Name of the column with preprocessed text.
        ngram_range (tuple): Range of n-grams to compute (e.g., (2, 3) for bi-grams and tri-grams).
        max_features (int): Maximum number of n-gram features.

    Returns:
        tuple: Sparse matrix of n-grams and fitted CountVectorizer.
    """
    # Compute n-grams using CountVectorizer
    vectorizer = CountVectorizer(ngram_range=ngram_range, max_features=max_features)
    ngram_matrix = vectorizer.fit_transform(df[text_column])

    return ngram_matrix, vectorizer

In [22]:
# Function to compute BERT embeddings for a single text
def bert_embedding(text, tokenizer, model):
    """
    Compute BERT embeddings for a single text.

    Args:
        text (str): Input text.
        tokenizer (BertTokenizer): Tokenizer for BERT.
        model (BertModel): Pre-trained BERT model.

    Returns:
        np.array: Sentence embedding (average of token embeddings).
    """
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=128, padding=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()

# Parallelized function to compute BERT embeddings
def compute_bert_embeddings_parallel(df, text_column, max_workers=4):
    """
    Compute BERT embeddings for all texts in a DataFrame using ThreadPoolExecutor.

    Args:
        df (pd.DataFrame): DataFrame containing the text column.
        text_column (str): Name of the column with text to embed.
        max_workers (int): Maximum number of threads.

    Returns:
        list: List of BERT embeddings.
    """
    texts = df[text_column].tolist()
    embeddings = []

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit tasks to the executor
        futures = {executor.submit(bert_embedding, text, tokenizer, model): text for text in texts}

        # Use tqdm for progress bar
        for future in tqdm(as_completed(futures), total=len(futures), desc="Computing BERT embeddings"):
            try:
                embeddings.append(future.result())
            except Exception as e:
                print(f"Error processing text: {e}")
                embeddings.append(None)  # Append None for failed embeddings

    return torch.tensor([emb for emb in embeddings if emb is not None])

In [23]:
def load_glove_embeddings(file_path):
    """
    Load GloVe embeddings into a dictionary.

    Args:
        file_path (str): Path to the GloVe file.

    Returns:
        dict: A dictionary mapping words to their vector embeddings.
    """
    embeddings = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            embeddings[word] = vector
    return embeddings

In [24]:
def get_embedding(word, embeddings_dict, embedding_dim=300):
    """
    Retrieve the embedding for a given word or a zero vector if the word is not in the embeddings.

    Args:
        word (str): Input word.
        embeddings_dict (dict): Dictionary of pre-trained embeddings.
        embedding_dim (int): Dimension of the embeddings.

    Returns:
        np.array: Embedding vector for the word.
    """
    return embeddings_dict.get(word, np.zeros(embedding_dim))

In [25]:
def compute_sentence_embedding(sentence, embeddings_dict, embedding_dim=300):
    """
    Compute sentence embedding by averaging word embeddings.

    Args:
        sentence (str): Input sentence.
        embeddings_dict (dict): Dictionary of pre-trained embeddings.
        embedding_dim (int): Dimension of the embeddings.

    Returns:
        np.array: Sentence embedding.
    """
    tokens = sentence.split()  # Tokenize the sentence
    token_embeddings = [get_embedding(token, embeddings_dict, embedding_dim) for token in tokens]
    return np.mean(token_embeddings, axis=0) if token_embeddings else np.zeros(embedding_dim)

In [26]:
# plotting confusion matrix
def plot_normalized_confusion_matrix(y_true, y_pred, class_labels, title="Confusion Matrix", cmap="Blues"):
    """
    Plots a normalized confusion matrix (percentages).

    Args:
        y_true (array-like): Ground truth labels.
        y_pred (array-like): Predicted labels.
        class_labels (list): List of class label names.
        title (str): Title of the confusion matrix plot.
        cmap (str): Colormap for the heatmap.
    """
    # Compute the confusion matrix
    cm = confusion_matrix(y_true, y_pred, labels=class_labels)

    # Normalize the confusion matrix by row (true class)
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100

    # Plot the normalized confusion matrix
    fig, ax = plt.subplots(figsize=(8, 6))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm_normalized, display_labels=class_labels)
    disp.plot(cmap=cmap, ax=ax, colorbar=True)

    plt.title(f'{title} (in %)')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.show()

In [27]:
def train_and_evaluate_pipeline(pipeline, X_train, y_train, X_val, y_val, X_test, y_test, label_encoder):
    """
    Trains a pipeline, evaluates it on validation and test sets, and prints performance metrics.

    Args:
        pipeline (Pipeline): A scikit-learn pipeline containing the model and preprocessing steps.
        X_train (array-like): Training feature set.
        y_train (array-like): Training labels.
        X_val (array-like): Validation feature set.
        y_val (array-like): Validation labels.
        X_test (array-like): Test feature set.
        y_test (array-like): Test labels.
        label_encoder (LabelEncoder): Fitted label encoder for decoding labels.

    Returns:
        dict: A dictionary containing validation accuracy, test accuracy, and the test classification report.
    """
    # Train the model
    pipeline.fit(X_train, y_train)

    # Evaluate on validation set
    y_val_pred_encoded = pipeline.predict(X_val)
    val_accuracy = accuracy_score(y_val, y_val_pred_encoded)

    # Evaluate on test set
    y_test_pred_encoded = pipeline.predict(X_test)
    y_test_pred = label_encoder.inverse_transform(y_test_pred_encoded)  # Decode predictions
    y_test_decoded = label_encoder.inverse_transform(y_test)  # Decode actual labels

    # Compute test accuracy
    test_accuracy = accuracy_score(y_test_decoded, y_test_pred)

    # Print results
    print(f"Validation Accuracy: {val_accuracy:.2f}")
    print(f"Test Accuracy: {test_accuracy:.2f}")
    print("Test Classification Report:")
    test_report = classification_report(y_test_decoded, y_test_pred, target_names=label_encoder.classes_)
    print(test_report)

    plot_normalized_confusion_matrix( y_true=y_test_decoded, y_pred=y_test_pred, \
                                     class_labels=label_encoder.classes_, \
                                      title="Confusion Matrix on Test Set")
    return {
        'validation_accuracy': val_accuracy,
        'test_accuracy': test_accuracy,
        'test_report': test_report
    }, pipeline

In [28]:
def fitness_function(selected_features, X, y, model, cv=3):
    """
    Fitness function to evaluate feature subset.

    Args:
        selected_features (list): Binary list representing selected features.
        X (array-like): Feature matrix.
        y (array-like): Target labels.
        model (sklearn model): Classifier to evaluate.
        cv (int): Number of cross-validation folds.

    Returns:
        float: Negative cross-validation accuracy (to minimize).
    """
    # Mask selected features
    X_selected = X[:, selected_features == 1]

    if X_selected.shape[1] == 0:  # Avoid empty feature set
        return 1.0  # High error for invalid subsets

    # Perform cross-validation and return negative accuracy
    score = cross_val_score(model, X_selected, y, cv=cv, scoring='accuracy').mean()
    return -score  # Negate because optimization minimizes

In [29]:
# Initialize GWO population
def initialize_population(n_wolves, n_features):
    return np.random.randint(2, size=(n_wolves, n_features))

# Update wolves with GWO
def update_gwo(population, alpha, beta, delta, a):
    n_wolves, n_features = population.shape
    new_population = np.copy(population)

    for i in range(n_wolves):
        for j in range(n_features):
            r1, r2 = rand(), rand()
            A = 2 * a * r1 - a
            C = 2 * r2
            D_alpha = abs(C * alpha[j] - population[i, j])
            X1 = alpha[j] - A * D_alpha

            D_beta = abs(C * beta[j] - population[i, j])
            X2 = beta[j] - A * D_beta

            D_delta = abs(C * delta[j] - population[i, j])
            X3 = delta[j] - A * D_delta

            # Update position
            new_population[i, j] = (X1 + X2 + X3) / 3

    return (new_population > 0.5).astype(int)

# Hybrid optimization (GWO + ABC)
def hybrid_gwo_abc(X, y, model, n_wolves=10, n_iter=20):
    n_features = X.shape[1]
    population = initialize_population(n_wolves, n_features)

    alpha, beta, delta = None, None, None
    a = 2  # Linear reduction coefficient

    for t in range(n_iter):
        fitness = np.array([fitness_function(wolf, X, y, model) for wolf in population])

        # Sort wolves by fitness
        sorted_indices = np.argsort(fitness)
        population = population[sorted_indices]

        # Update alpha, beta, delta
        alpha, beta, delta = population[0], population[1], population[2]

        # Update positions using GWO
        population = update_gwo(population, alpha, beta, delta, a)

        # Apply local ABC exploitation to the top wolves (e.g., top 3)
        for i in range(3):
            local_search_wolf = np.copy(population[i])
            for _ in range(randint(1, 5)):  # Random local updates
                feature_idx = randint(0, n_features)
                local_search_wolf[feature_idx] = 1 - local_search_wolf[feature_idx]  # Flip feature
            # Accept if fitness improves
            if fitness_function(local_search_wolf, X, y, model) < fitness[i]:
                population[i] = local_search_wolf

        # Reduce exploration coefficient
        a -= 2 / n_iter

    # Return the best solution
    best_wolf = population[0]
    return best_wolf

## 2.5 Data Loading

This section focuses on loading the COVID-19 Twitter dataset into a Pandas DataFrame for analysis. We will perform the following steps:

1. **Data Directory Determination:** Identify the appropriate directory where the data files are stored, considering both local and cloud (Colab) environments.

2. **File Identification:** Locate all CSV files within the determined data directory using the `glob` library.

3. **Data Loading and Concatenation:**
    - Read each CSV file into a separate Pandas DataFrame using `pd.read_csv`.
    - Concatenate all the individual DataFrames into a single DataFrame named `data` using `pd.concat`.
    - Print information about the loaded data, including its dimensions and a preview of the first few rows.

4. **Error Handling:** Implement error handling mechanisms to address potential issues during file loading, such as missing files or incorrect file formats.

In [30]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
## Loading the files
# determine the data directory
# data_dir would be global variable
data_dir = determine_data_dir()

# locate all CSV files in the determined directory
files_pattern = os.path.join(data_dir, "*.csv")
files = glob.glob(files_pattern)

# check if files are found
if not files:
    print(f"No CSV files found in directory: {data_dir}")
else:
     # load and inspect each file
    dfs = []  # to store valid DataFrames
    for file in files:
        try:
            # load the DataFrame
            df = pd.read_csv(file)
            rows, cols = df.shape
            print(f"File: {file} | Rows: {rows}, Columns: {cols}")

            # skip empty files or files with no columns
            if rows == 0 or cols == 0:
                print(f"Skipping empty or invalid file: {file}")
                continue

            # append to list if valid
            dfs.append(df)

        except Exception as e:
            print(f"Error loading file {file}: {e}")

    # concatenate all valid DataFrames
    if dfs:
        data = pd.concat(dfs, ignore_index=True)
        print(f"Data loaded successfully with {data.shape[0]} rows and {data.shape[1]} columns.")
        print(data.head())
    else:
        print("No valid DataFrames to concatenate.")

Running in Google Colab. Using data directory: /content/drive/MyDrive/Colab_Notebooks/Data
File: /content/drive/MyDrive/Colab_Notebooks/Data/Covid-19 Twitter Dataset (Aug-Sep 2020).csv | Rows: 120509, Columns: 17


In [ ]:
# loading GloVe embeddings (large file, might take a few minutes)
download_and_extract_glove(data_dir)

## 2.6 Basic Data Understanding

This section focuses on gaining an initial understanding of the dataset's structure and contents.
We will perform the following steps:
1. **Data Overview:** Examine the basic information about the dataset, including the number of rows, columns, and data types.
2. **Column Selection:** Identify and select the relevant columns for the analysis.


In [ ]:
# get general info about the dataset
data.info()

In [ ]:
# info about the numeric columns
data.describe()

In [ ]:
# select subset of tweets we'll be working with:
cols_to_keep = ['id', 'source', 'created_at', 'original_text', \
                'lang', 'favorite_count', 'retweet_count', 'original_author', \
                'hashtags', 'user_mentions', 'place', 'sentiment', 'compound', 'pos', 'neu', 'neg']
tweets_df = data[cols_to_keep].copy()
# drop NaN ids
tweets_df.dropna(subset=['id'], inplace=True)

In [ ]:
display_categorical_vals(tweets_df)

## 2.7 Data Cleaning: Date

In [ ]:
#rename date column for clarity and convert to date
tweets_df.rename(columns={'created_at': 'date'}, inplace=True)
tweets_df['date'] = pd.to_datetime(tweets_df['date'])

Follow by exploratory data analysis:
* What were daily tweet patters?
* What were the top 20 days with most tweets?  

In [ ]:
# exploratory analysis: plot number of tweets per day
# group by date and count tweets
tweets_per_day = tweets_df.groupby(tweets_df['date'].dt.date)['id'].count()

plt.figure(figsize=(12, 6))
plt.bar(tweets_per_day.index, tweets_per_day.values)
plt.xlabel("Date")
plt.ylabel("Number of Tweets")
plt.title("Number of Tweets Per Day")
plt.grid(True)
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
plt.tight_layout()  # Adjust layout to prevent labels from overlapping
plt.show()

In [ ]:
# exploratory analysis: list the top 20 days with the most tweets
# rename columns for clarity
tweets_per_day.columns = ['date', 'tweet_count']

# sort by tweet count and get the top 20
top_20_days = tweets_per_day.sort_values(ascending=False).head(20)

# display the result
print(top_20_days)

## 2.8 Data Cleaning: Language
Before coming up with the strategy for each column, we'll check the contents of categorical data and the distributiuon of NaNs.

* It would make sence that fields like ```hashtags``` and ```user_mentions``` would have missing values and we'll leave it as it is.
* We'll check the ```lang``` and ```place``` columns.


In [ ]:
# number of NaNs in lang
sum(tweets_df.lang.isna())

In [ ]:
# look at the tweet text
tweets_df[tweets_df.lang.isna()]

The only 3 rows where language is missing are missing the original text, so we'll discard them.

In [ ]:
# drop rows where 'lang' is NaN
tweets_df = tweets_df.dropna(subset=['lang'])

# verify the changes
print(f"Number of NaNs in 'lang' after dropping: {sum(tweets_df.lang.isna())}")

# drop the lang column from the df
tweets_df = tweets_df.drop(columns=['lang'])

In [ ]:
tweets_df

In [ ]:
#rename place column to location for clarity
tweets_df.rename(columns={'place': 'location'}, inplace=True)

In [ ]:
# count of NaNs
sum(tweets_df.location.isna())

In [ ]:
# fill location NaNs with Unknown
tweets_df.fillna(value={'location':'Unknown'}, inplace=True)

## 2.9 Data Cleaning: Location

Here we'll see if location can be cleaned up for further visualization. We'll start with checking the locations with at least 10 tweets.  

In [ ]:
# calculate value counts for the 'location' column
location_counts = tweets_df.location.value_counts()

# filter to keep locations with at least 20 occurrences
filtered_locations = location_counts[location_counts >= 10]

# fisplay the filtered results
print(filtered_locations)

Next, we'll check the locations with the special characters (non-alphanumeric that are not the ```-,.```).

In [ ]:
# apply the function to create a boolean mask
special_chars_mask = tweets_df['location'].apply(has_special_chars)

# filter the DataFrame and get value counts
locations_with_special_chars = tweets_df.loc[special_chars_mask, \
                                             'location'].value_counts()

# display the result
print(locations_with_special_chars)

 We'll try to salvage the locations by using the first or the last word in a multi-word location. Otherwise, set it to the 'Unknown'.

In [ ]:
# create a boolean mask for one-word locations, those we won't be able to automatically ID
one_word_mask = locations_with_special_chars.index.str.split().str.len() == 1
one_word_locations = locations_with_special_chars[one_word_mask].index

# replace those locations in the original DataFrame with 'Unknown'
tweets_df['location'] = tweets_df['location'].replace(one_word_locations, 'Unknown')

The brief explanation for ```geocode_location()``` function:
* It utilizes the geocoding API. We give it a location (string), and it returns a best(first) guess for the location. For example, if I give it ```Paris```, it will return ```Paris, France``` and not ```Paris, Texas```.
* It executes slowly.

In [ ]:
# test it on few entries
print(geocode_location('New York, NY'))             # Expected: "New York, New York, United States"
print(geocode_location('Toronto, Ontario'))         # Expected: "Toronto, Ontario, Canada"
print(geocode_location('India'))                    # Expected: "Unknown, Unknown, India"
print(geocode_location('USA'))                      # Expected: "Unknown, Unknown, United States"

Additionally, we implemented parallel optimization in ```batch_geocode()```.

In [ ]:
# before applying, do minor cleaning: replace 'unknown' with 'Unknown'
tweets_df.location.replace('unknown', 'Unknown')

In [ ]:
# calculate value counts for the 'location' column
location_counts = tweets_df.location.value_counts()

# filter to keep locations with at least 10 occurrences
filtered_locations = location_counts[location_counts >= 10]

# remove 'Unknown' from filtered_locations
filtered_locations_known = filtered_locations[filtered_locations.index != 'Unknown']

# create a boolean mask for locations in filtered_locations_known
mask = tweets_df['location'].isin(filtered_locations_known.index)

# get unique locations from the filtered DataFrame
unique_locations = tweets_df.loc[mask, 'location'].unique()

The following code will apply batch geocoding requests. To respect the Nominatim API, we put a sleep(0.5) before each request and longer timeout (8s).
The code would run ~40 minutes.

In [ ]:
# apply geocode_location only to the selected locations in parallel
geocoded_results = dict(zip(unique_locations, batch_geocode(unique_locations)))

# map the results back to the DataFrame
tweets_df.loc[mask, 'geocoded_location'] = tweets_df.loc[mask, 'location'].map(geocoded_results)

In [ ]:
# save location cache after processing
save_cache_to_json(location_cache)

In [ ]:
# check value counts
tweets_df['geocoded_location'].value_counts()

Continue location data cleaning. For all undecoded ```geocoded_location```, try to apply first then last words of ```location```.

In [ ]:
# create a mask for rows where 'geocoded_location' is 'Unknown' and where its known
unknown_mask = tweets_df['geocoded_location'] == 'Unknown'
known_mask = tweets_df['geocoded_location'] != 'Unknown'
# extract FIRST word from 'location' for 'Unknown' geocoded rows and batch process
if unknown_mask.any():
    first_words = tweets_df.loc[unknown_mask, 'location'].apply(lambda loc: extract_word(loc, position="first"))

    unique_first_words = first_words.unique()
    unique_first_words_set = set(unique_first_words)
    # extract known locations where geocoded_location is not 'Unknown'
    known_locations = tweets_df.loc[known_mask, 'location']
    # tokenize each location into words and find intersection with unique_first_words_set
    known_words_in_locations = known_locations.str.split().apply(lambda words: unique_first_words_set.intersection(words))

    # extract matches
    matched_words = {word for words in known_words_in_locations for word in words}

    # perform geocoding for matched first words
    first_word_results = batch_geocode(matched_words)
    # create a mapping of unique first words to geocoded results
    first_word_mapping = dict(zip(matched_words, first_word_results))

    # map the results back to the DataFrame
    tweets_df.loc[unknown_mask, 'geocoded_location'] = first_words.map(first_word_mapping)

In [ ]:
# update the mask for remaining 'Unknown' rows
unknown_mask = tweets_df['geocoded_location'] == 'Unknown'
known_mask = tweets_df['geocoded_location'] != 'Unknown'

# extract LAST word from 'location' for remaining 'Unknown' rows and batch process
if unknown_mask.any():
    last_words = tweets_df.loc[unknown_mask, 'location'].apply(lambda loc: extract_word(loc, position="last"))
    unique_last_words = last_words.unique()
    unique_last_words_set = set(last_words)

    known_locations = tweets_df.loc[known_mask, 'location']

     # tokenize each location into words and find intersection with unique_first_words_set
    known_words_in_locations = known_locations.str.split().apply(lambda words: unique_last_words_set.intersection(words))

    # extract matches
    matched_words = {word for words in known_words_in_locations for word in words}

    # perform geocoding for unique last words
    last_word_results = batch_geocode(matched_words)  # Ensure batch_geocode is defined
    # create a mapping of unique last words to geocoded results
    last_word_mapping = dict(zip(matched_words, last_word_results))

    # map the results back to the DataFrame
    tweets_df.loc[unknown_mask, 'geocoded_location'] = last_words.map(last_word_mapping)

In [ ]:
# if still 'Unknown', give up
unknown_mask = tweets_df['geocoded_location'] == 'Unknown'
if unknown_mask.any():
    print(f"Giving up on {unknown_mask.sum()} locations. Could not geocode these entries.")

In [ ]:
# update location cache after processing
save_cache_to_json(location_cache)

In [ ]:
# fill location NaNs with Unknown
tweets_df.fillna(value={'geocoded_location':'Unknown'}, inplace=True)

Next, we'll use ```split_geocoded_location()``` to split the geocoded_location into three columns: ```country```, ```state```, and ```city```.

In [ ]:
# apply the function to create separate columns
tweets_df[['city', 'state', 'country']] = tweets_df['geocoded_location'].apply(
    lambda loc: pd.Series(split_geocoded_location(loc))
)
# display the DataFrame with new columns
tweets_df[['geocoded_location', 'city', 'state', 'country']]

In [ ]:
# minor fix: capitalization
# Capitalize all entries in 'City', 'State', and 'Country' columns
tweets_df['city'] = tweets_df['city'].str.title()
tweets_df['state'] = tweets_df['state'].str.title()
tweets_df['country'] = tweets_df['country'].str.title()

Next we'll perform location EDA: tweet histograms by Country (Worldwide), by City (Worldwide), By  State(US), followed by additional cleaning.

In [ ]:
# 1. Country EDA: Remove 'Unknown' countries and plot histogram for top 20 (ordered)
known_countries = tweets_df[tweets_df['country'] != 'Unknown']
top_20_countries = known_countries['country'].value_counts().nlargest(20)

plt.figure(figsize=(12, 6))
sns.barplot(
    y=top_20_countries.index,
    x=top_20_countries.values,
    hue=top_20_countries.index,  # Assign y to hue
    palette='viridis',
    dodge=False,  # Ensure no splitting
    legend=False  # Disable legend
)
plt.title('Distribution of Tweets by Top 20 Countries (Excluding Unknown)')
plt.xlabel('Number of Tweets')
plt.ylabel('Country')
for i, v in enumerate(top_20_countries.values):
    plt.text(v, i, f"{v}", va='center')
plt.show()

In [ ]:
# 2. Remove 'Unknown' cities and plot histogram for top 50 (ordered)
known_cities = tweets_df[tweets_df['city'] != 'Unknown']
top_50_cities = known_cities['city'].value_counts().nlargest(50)

plt.figure(figsize=(12, 10))
sns.barplot(
    y=top_50_cities.index,
    x=top_50_cities.values,
    hue=top_50_cities.index,  # Assign y to hue
    palette='viridis',
    dodge=False,  # Ensure no splitting
    legend=False  # Disable legend
)
plt.title('Distribution of Tweets by Top 50 Cities (Excluding Unknown)')
plt.xlabel('Number of Tweets')
plt.ylabel('City')
for i, v in enumerate(top_50_cities.values):
    plt.text(v, i, f"{v}", va='center')
plt.show()

In [ ]:
# Filter out 'Unknown' states and keep only US entries
us_states = tweets_df[(tweets_df['country'] == 'United States') & (tweets_df['state'] != 'Unknown')]

# Count occurrences of each state and get the top 20
state_counts = us_states['state'].value_counts().nlargest(20)

# Plot the distribution of tweets by the top 20 US states
plt.figure(figsize=(12, 6))
sns.barplot(
    y=state_counts.index,
    x=state_counts.values,
    hue=state_counts.index,  # Use state as hue
    palette='viridis',
    dodge=False,  # Prevent splitting bars
    legend=False  # Suppress the legend
)
plt.title('Distribution of Tweets by Top 20 US States (Excluding Unknown)')
plt.xlabel('Number of Tweets')
plt.ylabel('US State')

# Add labels to the bars
for i, v in enumerate(state_counts.values):
    plt.text(v, i, f"{v}", va='center')

plt.show()

In [ ]:
# minor fixes: Move Florida, California, Texas, Michigan  from City to State and rename None in state to Unknown
states_as_cities = ['Florida', 'California', 'Texas', 'Michigan']
mask_states_as_cities = (tweets_df['city'].isin(states_as_cities)) & (tweets_df['country'] == 'United States')

# update State and City columns for these entries
tweets_df.loc[mask_states_as_cities, 'state'] = tweets_df.loc[mask_states_as_cities, 'city']
tweets_df.loc[mask_states_as_cities, 'city'] = 'Unknown'

# rename all 'None' in State to 'Unknown'
tweets_df['state'] = tweets_df['state'].replace('None', 'Unknown')

# fix "Alba/Scotland" entries
mask_alba_scotland = tweets_df['city'].str.contains('Alba/Scotland', case=False, na=False)

# update City and Country for Alba/Scotland
tweets_df.loc[mask_alba_scotland, 'city'] = 'Unknown'
tweets_df.loc[mask_alba_scotland, 'country'] = 'Scotland'

In [ ]:
# redo the city plot:
# 2. Remove 'Unknown' cities and plot histogram for top 50 (ordered)
known_cities = tweets_df[tweets_df['city'] != 'Unknown']
top_50_cities = known_cities['city'].value_counts().nlargest(50)

plt.figure(figsize=(12, 10))
sns.barplot(
    y=top_50_cities.index,
    x=top_50_cities.values,
    hue=top_50_cities.index,  # Assign y to hue
    palette='viridis',
    dodge=False,  # Ensure no splitting
    legend=False  # Disable legend
)
plt.title('Distribution of Tweets by Top 50 Cities (Excluding Unknown)')
plt.xlabel('Number of Tweets')
plt.ylabel('City')
for i, v in enumerate(top_50_cities.values):
    plt.text(v, i, f"{v}", va='center')
plt.show()

In [ ]:
# redo the states plot:
# Filter out 'Unknown' states and keep only US entries
us_states = tweets_df[(tweets_df['country'] == 'United States') & (tweets_df['state'] != 'Unknown')]

# Count occurrences of each state and get the top 20
state_counts = us_states['state'].value_counts().nlargest(20)

# Plot the distribution of tweets by the top 20 US states
plt.figure(figsize=(12, 6))
sns.barplot(
    y=state_counts.index,
    x=state_counts.values,
    hue=state_counts.index,  # Use state as hue
    palette='viridis',
    dodge=False,  # Prevent splitting bars
    legend=False  # Suppress the legend
)
plt.title('Distribution of Tweets by Top 20 US States (Excluding Unknown)')
plt.xlabel('Number of Tweets')
plt.ylabel('US State')

# Add labels to the bars
for i, v in enumerate(state_counts.values):
    plt.text(v, i, f"{v}", va='center')

plt.show()

In [ ]:
# USA tweets per day
usa_tweets = tweets_df[tweets_df['country'] == 'United States']

# group by date and count tweets per day
tweets_per_day = usa_tweets.groupby(usa_tweets['date'].dt.date).size().reset_index(name='Total Tweets')
tweets_per_day.columns = ['Date', 'Total Tweets']

# bar plot
plt.figure(figsize=(14, 6))
sns.barplot(
    data=tweets_per_day,
    x='Date',
    y='Total Tweets',
    hue='Date',  # Assign the x variable to hue
    palette='viridis',
    legend=False  # Disable the legend
)

# customize x-axis ticks to show every third date
xticks = plt.gca().get_xticks()
xtick_labels = tweets_per_day['Date'].astype(str).values
plt.xticks(
    ticks=xticks[::3],  # show every third tick
    labels=xtick_labels[::3],  # use corresponding labels
    rotation=90  # rotate 45 deg for better visibility
)

plt.xticks(rotation=90)  # rotate x-axis labels for better readability
plt.title('Tweets Per Day in the USA')
plt.xlabel('Date')
plt.ylabel('Number of Tweets')
plt.tight_layout()  # Adjust layout to prevent label overlap
plt.show()

**[Optional] Adding lattitude, longitude (long runtime)**. This piece of code will run for about 1 hr, it's going to retrieve geo coordinates (latitude, longitude) for all cities in the dataset. Strongly recommend executing the following ```save_cache_to_json()``` to save the work and downloading the file if you're running the notebook from the cloud.

For this particular dataset or scope of the business problem it might not worth it, but for advanced analysis or futire work it might be a useful EDA section to utilize.  

In [ ]:
tweets_df = add_coordinates_with_progress(tweets_df)

In [ ]:
# save coordinate cache to json
save_cache_to_json(coordinate_cache, "coordinate_cache.json")

In [ ]:
# Generate the map if it's not found
try:
    # Try to open the file
    heatmap_file = os.path.join(data_dir, "heatmap_city.html")
    with open(heatmap_file, "r") as file:
        print("heatmap_city.html already exists. No need to recompute.")
        heatmap_city = file.read()
except FileNotFoundError:
    print("heatmap.html does not exist. Generating the heatmap...")
    # Call the function to compute the heatmap
    # Filter rows where both latitude and longitude are not 0.0
    filtered_tweets_df = tweets_df[(tweets_df['latitude'] != 0.0) & (tweets_df['longitude'] != 0.0)]
    # Create tweet_count column
    filtered_tweets_df.loc[:, 'tweet_count'] = filtered_tweets_df.groupby('city')['city'].transform('count')
    heatmap_city = generateBaseMap(input_type="city", df=filtered_tweets_df)
    heatmap_city.save(heatmap_file)

In [ ]:
# Display the map directly in the notebook (might not work)
display(HTML(heatmap_city))

## 2.10 Data Cleaning: Source
Next, we'll replace the ```source``` values with readable meaningful values using ```extract_html_source()``` function that will extract the value between the HTML tags.

In [ ]:
# replace the source with the meaningful value
tweets_df.source.value_counts()

In [ ]:
# apply the function to the 'source' column
tweets_df['source'] = tweets_df['source'].apply(extract_html_source)
# identify sources with counts less than 100
source_counts = tweets_df['source'].value_counts()
low_count_sources = source_counts[source_counts < 100].index

# replace low-count sources with 'Other'
tweets_df['source'] = tweets_df['source'].replace(low_count_sources, 'Other')
#replace NaNs with 'Other'
tweets_df['source'].fillna('Other', inplace=True)
tweets_df['source'].value_counts()

In [ ]:
# plot the top 20 sources
top_20_sources = tweets_df['source'].value_counts().nlargest(20)

# Convert to a DataFrame for plotting
top_20_df = top_20_sources.reset_index()
top_20_df.columns = ['Source', 'Count']

# Plot the histogram
plt.figure(figsize=(12, 6))
sns.barplot(
    data=top_20_df,
    y='Source',
    x='Count',
    hue='Source',  # Assign the x variable to hue
    palette='viridis',
    legend=False  # Disable the legend
)
plt.title('Top 20 Twitter Sources')
plt.xlabel('Number of Tweets')
plt.ylabel('Source')
plt.tight_layout()
plt.show()

## 2.11 Data Cleaning: Sentiment

In [ ]:
# Show distribution of tweet sentiments
sentiment_counts = tweets_df.sentiment.value_counts()
print(sentiment_counts)
# is there any NaNs
print(tweets_df.sentiment.isna().sum())

In [ ]:
# Define custom colors for the sentiments
custom_colors = {
    'pos': '#90EE90',  # Light Green
    'neu': '#ADD8E6',   # Light Blue
    'neg': '#FFB6C1'   # Light Red
}

# Ensure the colors map correctly to the sentiment categories
print("Sentiment Categories:", sentiment_counts.index)  # Debugging step
colors = [custom_colors.get(sentiment, '#D3D3D3') for sentiment in sentiment_counts.index]

# Check the colors being applied
print("Applied Colors:", colors)  # Debugging step

# Plot the pie chart
plt.figure(figsize=(8, 8))
plt.pie(
    sentiment_counts.values,
    labels=sentiment_counts.index,
    autopct=lambda p: f'{p:.1f}% ({int(p * sum(sentiment_counts.values) / 100)})',  # Percentage and count
    colors=colors,  # Use custom colors
    startangle=90
)
plt.title('Sentiment Distribution', fontsize=14)
plt.show()

In [ ]:
# Define sentiments and their corresponding colors
sentiments = ['pos', 'neu', 'neg']
colors = ['lightgreen', 'lightblue', 'lightcoral']

# Scatter plots for the relationship between sentiment scores and compound value
plt.figure(figsize=(15, 5))

for i, (sentiment, color) in enumerate(zip(sentiments, colors), start=1):
    plt.subplot(1, 3, i)
    sns.scatterplot(
        data=tweets_df,
        x=sentiment,
        y='compound',
        alpha=0.6,
        color=color
    )
    plt.title(f'Compound Score vs {sentiment.capitalize()}')
    plt.xlabel(sentiment.capitalize())
    plt.ylabel('Compound Score')

plt.tight_layout()
plt.show()

From Kaggle's data card:
Algorithm Sentiment Classification of Tweets (compound, sentiment):

if tweet[compound] < 0:
tweet[sentiment] = 0.0 # assigned 0.0 for Negative Tweets
elif tweet[compound] > 0:
tweet[sentiment] = 1.0 # assigned 1.0 for Positive Tweets
else:
tweet[sentiment] = 0.5 # assigned 0.5 for Neutral Tweets
end```

In [ ]:
pd.reset_option('display.max_colwidth')

# Filter the dataset for rows where sentiment is 'pos'
positive_tweets = tweets_df[tweets_df['sentiment'] == 'pos']

# Randomly select 5 rows and specific columns
subset = positive_tweets[['original_text', 'pos', 'neu', 'neg', 'compound']].sample(n=5, random_state=42)
subset

In [ ]:
# Filter the dataset for rows where sentiment is 'pos'
neutral_tweets = tweets_df[tweets_df['sentiment'] == 'neu']

# Randomly select 5 rows and specific columns
subset = neutral_tweets[['original_text', 'pos', 'neu', 'neg', 'compound']].sample(n=5, random_state=42)
subset

In [ ]:
# Filter the dataset for rows where sentiment is 'pos'
negative_tweets = tweets_df[tweets_df['sentiment'] == 'neg']

# Randomly select 5 rows and specific columns
subset = negative_tweets[['original_text', 'pos', 'neu', 'neg', 'compound']].sample(n=5, random_state=42)
subset

Scatterplot for value distribution in neutral sentiment column.

In [ ]:
tweets_df['neu'].value_counts()

Understanding sentiment compound distribution between positive, neutral, negative score values.

Here we have a good distribution for positive, negative tweets. We're unable to plot neutral tweets's distribution because the variance is almost zero (most variables are around 0).

In [ ]:
plt.figure(figsize=(12, 6))
sns.kdeplot(
    data=tweets_df,
    x='compound',
    hue='sentiment',
    fill=True,
    alpha=0.6,
    palette={'neg': 'lightcoral', 'neu': 'lightblue', 'pos': 'lightgreen'}
)
plt.title('KDE of Compoundby by Sentiment')
plt.xlabel('Compound Weighted')
plt.ylabel('Density')
plt.show()

**Re-computing the compound score using weighted formula: Compound Score=(pos−neg)×(1−neu)**

We'll also exclude rows where all 3 values for `pos`, `neu`, `neg` are either 0 or 1.

In [ ]:
# Filter out rows where all three values are either 0 or 1
valid_rows = ~(
    ((tweets_df['pos'] == 0) & (tweets_df['neu'] == 0) & (tweets_df['neg'] == 0)) |
    ((tweets_df['pos'] == 1) & (tweets_df['neu'] == 1) & (tweets_df['neg'] == 1))
)

# Apply the filter explicitly with .loc[]
tweets_df = tweets_df.loc[valid_rows]

# Compute the weighted compound score using .loc[]
tweets_df.loc[:, 'compound_weighted'] = (tweets_df['pos'] - tweets_df['neg']) * (1 - tweets_df['neu'])

# Scale it to [0, 1]
tweets_df['compound_weighted'] = (tweets_df['compound_weighted'] - tweets_df['compound_weighted'].min()) / (
    tweets_df['compound_weighted'].max() - tweets_df['compound_weighted'].min()
)

# Display a preview of the updated DataFrame
print(tweets_df[['pos', 'neu', 'neg', 'compound', 'compound_weighted', 'sentiment']].head())

In [ ]:
# replot scores sentiments
sentiments = ['pos', 'neu', 'neg']
colors = ['lightgreen', 'lightblue', 'lightcoral']

# Scatter plots for the relationship between sentiment scores and compound value
plt.figure(figsize=(15, 5))

for i, (sentiment, color) in enumerate(zip(sentiments, colors), start=1):
    plt.subplot(1, 3, i)
    sns.scatterplot(
        data=tweets_df,
        x=sentiment,
        y='compound_weighted',
        alpha=0.6,
        color=color
    )
    plt.title(f'Compound Score vs {sentiment.capitalize()}')
    plt.xlabel(sentiment.capitalize())
    plt.ylabel('Compound Score')

plt.tight_layout()
plt.show()

In [ ]:
# Plotting new and old compound score distributions.
plt.figure(figsize=(16, 6))

# Plot weighted compound distribution
plt.subplot(1, 2, 1)
sns.histplot(tweets_df['compound_weighted'], bins=30, kde=False, color='purple')
plt.title('Distribution of Weighted Compound Scores')
plt.xlabel('Weighted Compound Score')
plt.ylabel('Frequency')

# Plot compound distribution
plt.subplot(1, 2, 2)
sns.histplot(tweets_df['compound'], bins=30, kde=False, color='orange')
plt.title('Distribution of Original Compound Scores')
plt.xlabel('Compound Score')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
# computing new sentiments:

# Define conditions and labels for new_simple_sentiment
simple_conditions = [
    (tweets_df['compound_weighted'] <= 0.49),  # Negative sentiment
    (tweets_df['compound_weighted'] >0.49) & (tweets_df['compound_weighted'] <= 0.51),  # Neutral sentiment
    (tweets_df['compound_weighted'] > 0.51)  # Positive sentiment
]
simple_labels = ['neg', 'neu', 'pos']

# Define conditions and labels for new_advanced_sentiment
advanced_conditions = [
    (tweets_df['compound_weighted'] < 0.35),  # Strongly Negative
    (tweets_df['compound_weighted'] >= 0.35) & (tweets_df['compound_weighted'] <= 0.49),  # Weakly Negative (neu-neg)
    (tweets_df['compound_weighted'] >0.49) & (tweets_df['compound_weighted'] <= 0.51),  # Neutral sentiment
    (tweets_df['compound_weighted'] > 0.51) & (tweets_df['compound_weighted'] <= 0.75),  # Weakly Positive (neu-pos)
    (tweets_df['compound_weighted'] > 0.75)  # Strongly Positive
]
advanced_labels = ['neg', 'neu-neg', 'neu', 'neu-pos', 'pos']

# Assign new_simple_sentiment
tweets_df['new_simple_sentiment'] = pd.cut(
    tweets_df['compound_weighted'],
    bins=[-float('inf'), 0.49, 0.51, float('inf')],  # Define boundaries
    labels=['neg', 'neu', 'pos']
)

# Assign new_advanced_sentiment
tweets_df['new_advanced_sentiment'] = pd.cut(
    tweets_df['compound_weighted'],
    bins=[-float('inf'), 0.35, 0.49, 0.51, 0.75, float('inf')],  # Define boundaries
    labels=['neg', 'neu-neg', 'neu', 'neu-pos', 'pos']
)

# Check the distribution of the new sentiment columns
print(tweets_df['new_simple_sentiment'].value_counts())
print(tweets_df['new_advanced_sentiment'].value_counts())

Intermediate conclusions:
* We don't know whow assigned scores are computed (it's not regular or weighted computation).
* We can guess the prescence of a weights placed on the negative or positive parts of the score, when one dominates the other. That would explain the difference in compound score distribution, skewing tweets with more positive or negative scores urther away from neutral.
* We should also consider that back in 2020 twitter was a better moderated environment, and many tweets would originate from official accounts, contributing to their neutral sentiment.  

## 2.12 Exploratory Data Analysis (EDA): Social Connections
Vanity fair. Lookng at social engagement, connections, popularity.

* Hashtags word cloud.

In [ ]:
# Flatten the list of hashtags
all_hashtags = [
    ''.join(hashtags)  # Join characters into a string
    for hashtags in tweets_df['hashtags']
    if isinstance(hashtags, (list, tuple, str, np.ndarray))  # Check if iterable
]
# Convert the list of hashtags into a single string, separated by spaces
hashtag_text = ' '.join(all_hashtags)

# Generate the word cloud
wordcloud = WordCloud(width=800, height=400, \
                      background_color='white', colormap='viridis').generate(hashtag_text)

# Display the word cloud
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title("Hashtag Word Cloud", fontsize=16)
plt.show()

* Histograms for favorite tweets, retweets.
Convert both to log scale because of the value range (wide range, outliers).


In [ ]:
# Log-transform favorite_count (add 1 to avoid log(0))
tweets_df['log_favorite_count'] = np.log1p(tweets_df['favorite_count'])

# Histogram for log-transformed favorite_count
plt.figure(figsize=(10, 5))
sns.histplot(tweets_df['log_favorite_count'], bins=30, kde=True, color='blue')
plt.title('Distribution of Log-Transformed Favorite Counts')
plt.xlabel('Log(Favorite Count)')
plt.ylabel('Frequency')
plt.show()

# Log-transform retweet_count (add 1 to avoid log(0))
tweets_df['log_retweet_count'] = np.log1p(tweets_df['retweet_count'])

# Histogram for log-transformed retweet_count
plt.figure(figsize=(10, 5))
sns.histplot(tweets_df['log_retweet_count'], bins=30, kde=True, color='green')
plt.title('Distribution of Log-Transformed Retweet Counts')
plt.xlabel('Log(Retweet Count)')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Log-transform favorite_count and retweet_count (add 1 to avoid log(0))
tweets_df['log_favorite_count'] = np.log1p(tweets_df['favorite_count'])
tweets_df['log_retweet_count'] = np.log1p(tweets_df['retweet_count'])

# Scatter plot with log-transformed values
plt.figure(figsize=(8, 6))
sns.scatterplot(x='log_favorite_count', y='log_retweet_count', data=tweets_df, alpha=0.6)
plt.title('Log(Favorites) vs Log(Retweets)')
plt.xlabel('Log(Favorite Count)')
plt.ylabel('Log(Retweet Count)')
plt.show()

In [ ]:
#  Show trends in favorites and retweets over time.
aggregated = tweets_df.groupby('date')[['favorite_count', 'retweet_count']].sum().reset_index()

plt.figure(figsize=(12, 6))
sns.lineplot(x='date', y='favorite_count', data=aggregated, label='Favorites', color='blue')
plt.yscale('log')
sns.lineplot(x='date', y='retweet_count', data=aggregated, label='Retweets', color='green')
plt.title('Log Trends in Favorites and Retweets Over Time')
plt.xlabel('Date')
plt.ylabel('Count')
plt.legend()
plt.show()

In [ ]:
# top contributors by the number of tweets
top_authors = tweets_df['original_author'].value_counts().head(10)
# Bar Plot for Top Authors
plt.figure(figsize=(10, 5))
sns.barplot(
    x=top_authors.values,
    y=top_authors.index,
    hue=top_authors.index,  # Assign hue to the y variable
    palette='viridis',
    dodge=False,            # Ensure no splitting
    legend=False            # Disable the legend
)
plt.title('Top 10 Authors by Number of Tweets')
plt.xlabel('Number of Tweets')
plt.ylabel('Author')
plt.show()

In [ ]:
# Plot authors based on their average favorite_count and retweet_count
author_stats = tweets_df.groupby('original_author')[['favorite_count', \
                                                     'retweet_count']].mean().reset_index()

author_stats['log_favorite_count'] = np.log1p(author_stats['favorite_count'])
author_stats['log_retweet_count'] = np.log1p(author_stats['retweet_count'])

plt.figure(figsize=(10, 6))
sns.scatterplot(
    x='log_favorite_count', y='log_retweet_count', data=author_stats, alpha=0.6
)
plt.title('Author Popularity: Log-Transformed Average Favorites vs Retweets')
plt.xlabel('Log(Average Favorite Count)')
plt.ylabel('Log(Average Retweet Count)')
plt.show()

* Visualize the most mentioned users.

In [ ]:
# Process user_mentions: split by comma and remove empty/malformed entries
tweets_df['user_mentions'] = tweets_df['user_mentions'].fillna('')  # Replace None with empty strings
tweets_df['user_mentions_clean'] = tweets_df['user_mentions'].apply(lambda x: [mention.strip() for mention in x.split(',') if mention.strip()])

# Flatten mentions and count frequencies
all_mentions = [mention for mentions in tweets_df['user_mentions_clean'] for mention in mentions]
mention_counts = Counter(all_mentions).most_common(10)

# Convert to DataFrame for visualization
mention_df = pd.DataFrame(mention_counts, columns=['user', 'count'])

# Plot the data
plt.figure(figsize=(10, 5))
sns.barplot(
    x='count',
    y='user',
    data=mention_df,
    hue='user',        # Assign hue to the y variable
    palette='magma',
    dodge=False,       # Prevent bar splitting
    legend=False       # Disable the legend
)
plt.title('Top 10 Mentioned Users', fontsize=14)
plt.xlabel('Number of Mentions', fontsize=12)
plt.ylabel('User', fontsize=12)
plt.tight_layout()  # Adjust layout for better spacing
plt.show()

* Show the relationships between authors and mentioned users. With a subset that big, we won't have computational resources to compute a full graph.  

In [ ]:
# Filter nodes with degree above a certain threshold (200 connections)
# Create a directed graph
G = nx.DiGraph()
# Populate the graph with edges from tweets_df
for _, row in tweets_df.iterrows():
    original_author = row['original_author']
    mentioned_users = row['user_mentions_clean']  # List of mentioned users

    # Add edges: author -> mentioned user
    for mention in mentioned_users:
        G.add_edge(original_author, mention)

degree_threshold = 200
high_degree_nodes = [node for node, degree in G.degree() if degree > degree_threshold]
subgraph = G.subgraph(high_degree_nodes)

# Draw the subgraph
plt.figure(figsize=(12, 12))
nx.draw_networkx(
    subgraph,
    node_size=50,
    alpha=0.7,
    font_size=8,
    edge_color='gray'
)
plt.title(f'User Mentions Network (Nodes with Degree > {degree_threshold})')
plt.show()


* Display correlation between different metrics (e.g., favorites, retweets, mentions).*italicized text*

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(tweets_df[['favorite_count', 'retweet_count']].corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Between Metrics')
plt.show()

* Visualize authors with bubbles representing their popularity.

In [ ]:
# Aggregate favorite_count and retweet_count for each author
author_stats = tweets_df.groupby('original_author').agg({
    'favorite_count': 'mean',
    'retweet_count': 'mean',
    'original_author': 'count'  # Count tweets for each author
}).rename(columns={'original_author': 'tweet_count'}).reset_index()

plt.figure(figsize=(10, 6))
sns.scatterplot(
    x='favorite_count',
    y='retweet_count',
    size='tweet_count',
    data=author_stats,
    sizes=(50, 500),
    alpha=0.7
)
plt.title('Author Popularity: Favorites vs Retweets with Tweet Count')
plt.xlabel('Average Favorite Count')
plt.ylabel('Average Retweet Count')
plt.legend(title='Tweet Count', loc='upper left', bbox_to_anchor=(1, 1))
plt.show()

#3. Text Preprocessing and Feature Engineering

In this section we'll be transforming the contents of the `original_tweet` column to the text format suitable for classification. This will include the following steps:

**Cleaning**

This step removes noise and unnecessary elements from the raw tweet text, such as special characters, links, mentions (@usernames), and hashtags (#topic), ii also detects emoji and converts it to plain text, (😀 to smile face).

**Preprocessing**
This section prepares the cleaned text for sentiment analysis by breaking it down into individual words (tokenization), removing common words that don't carry much meaning (stop word removal), and reducing words to their base form (lemmatization).
**Feature Extraction**

This stage involves creating numerical representations of the text that can be used as input for machine learning models. We'll be extracting
* Bag of Words (BoW): This technique represents text as a collection of individual words and their frequencies, ignoring grammar and word order. It creates a numerical vector for each document, where each element represents the count of a specific word in the document's vocabulary.
* N-grams: Creating combinations of words (e.g., "Covid cases" as a bigram, or "Die of Covid") to capture more context.
* TF-IDF: Calculating the importance of words in a document relative to a collection of documents.
* Embeddings: Using pre-trained models to create vector representations of words that capture semantic meaning.


## 3.1 Text Cleaning
We wrote ```process_tweet_data()```to do the following:
* Remove and store emoji as a separate column(using default UNICODE_EMOJI).
* Remove and store mentions @ as a separate column.
* Remove and store retweets RT @ as a separate column.
* Remove and store hashtags # as a separate column.
* Remove and store URLs www. or t. or bit. as a separate column.
* Remove special characters, whitespaces, numbers.
* Lowercase and store text in ```cleaned_text``` column  


For debugging purposes, we'll create a small dataframe with URLs, mentions, retweents, and emojis.

In [ ]:
# example df
df_test = {
    'tweet_text': [
        "WOW!!! Check out this link: https://example.com :) @user123 #hashtag #fun",
        "RT @user456: Another day in paradise! 😃 #sunnyday",
        "Why so serious? :(( Visit www.example.org for details! #serious"
    ]
}
df_test = pd.DataFrame(df_test)

# Clean the text data
df_test = pd.concat([
    df_test,
    df_test['tweet_text'].apply(lambda x: pd.Series(process_tweet_data(x)))
], axis=1)
df_test

We'll apply text cleaning with progress to the subset of entire twitter df (takes about 2 hours minutes)

In [ ]:
tweets_slice_df = tweets_df[['id', 'date', 'source', 'original_text', 'sentiment', \
                             'pos', 'neu', 'neg', 'compound', 'compound_weighted', \
                             'new_simple_sentiment', 'new_advanced_sentiment']].copy()

In [ ]:
# check for NaNs before text pre-processing.
tweets_slice_df.isna().sum()
# drop NaNs from ID
tweets_slice_df = tweets_slice_df.dropna(subset=['id'])
#re-check for NaNs:
tweets_slice_df.isna().sum()

In [ ]:
# Remove all retweets
# Detect retweets: Create a mask for rows that are retweets
retweet_mask = tweets_slice_df['original_text'].str.startswith('RT @')

# Filter out retweets
tweets_df_no_retweets = tweets_slice_df[~retweet_mask].copy()

# Print results
print(f"Original DataFrame size: {tweets_df.shape[0]} rows")
print(f"DataFrame size after removing retweets: {tweets_df_no_retweets.shape[0]} rows")

In [ ]:
# apply the function to each tweet and expand the results into separate columns
cleaned_text_column = clean_tweets_with_progress_parallel(tweets_df_no_retweets, text_col='original_text')

In [ ]:
# Add the cleaned text column to dataframe:
tweets_df_no_retweets['cleaned_text'] = cleaned_text_column

In [ ]:
tweets_df_no_retweets = tweets_df_no_retweets.dropna(subset=['cleaned_text'])
tweets_df_no_retweets.isna().sum()

In [ ]:
# save as inrermideate processing step
path_to_saved_file = os.path.join(data_dir, "tweets_df_no_retweets.csv")  # Path to saved file
tweets_df_no_retweets.to_csv(path_to_saved_file, index=False)
print("tweets_df_no_retweets has been saved as 'tweets_df_no_retweets.csv'.")

## 3.2 Text Preprocessing

This section focuses on preparing the tweet text data for sentiment analysis by applying essential preprocessing techniques. The following steps are performed:

* Tokenization: The text of each tweet is broken down into individual words or tokens using the word_tokenize function from the nltk library.

* Stop Word Removal: Common words (like "the," "a," "is") that don't carry much meaning are removed from the tokenized text to reduce noise and improve analysis accuracy.

* Lemmatization: Words are reduced to their base or root form (e.g., "running" becomes "run") using the WordNetLemmatizer to standardize the vocabulary and group similar words together.

* Text Recombination: The preprocessed tokens are combined back into a single text string for further analysis.

In [ ]:
# load tweets_slice_df if memory crashes
path_to_saved_file = os.path.join(data_dir, "tweets_df_no_retweets.csv")  # Path to saved file
tweets_df_no_retweets = pd.read_csv(path_to_saved_file, low_memory=False)
tweets_df_no_retweets.dropna(subset=['cleaned_text'], inplace=True)

In [ ]:
tweets_df_no_retweets.head()

In [ ]:
# Preprocess text in the 'cleaned_text' column
preprocessed_df_test = preprocess_text(df_test, text_column='cleaned_text')
# Display the preprocessed DataFrame
preprocessed_df_test[['cleaned_text', 'lemmatized_text']]

In [ ]:
# apply lemmatization to entire dataframe
lemmatized_df = preprocess_text(tweets_df_no_retweets, text_column='cleaned_text')

In [ ]:
lemmatized_df.isna().sum()

In [ ]:
lemmatized_df= lemmatized_df.dropna(subset=['lemmatized_text'])
# checking NaNs
lemmatized_df.isna().sum()

In [ ]:
# save lemmatized_df if memory crashes
path_to_saved_lemmatized_file = os.path.join(data_dir, "lemmatized_df.csv")  # Path to saved file
lemmatized_df.to_csv(path_to_saved_lemmatized_file, index=False)
print("lemmatized_df has been saved as 'lemmatized_df.csv'.")

## 3.3. Feature Extraction.
In this section, we'll transform the preprocessed text data into numerical representations suitable for machine learning models. We'll explore techniques like:

N-grams: Extracting sequences of adjacent words (e.g., "social distancing," "stay home") to capture contextual information.
TF-IDF: Calculating word importance based on their frequency within a document and across the entire corpus.
Embeddings (optional): Representing words as dense vectors to capture semantic relationships.
These extracted features will serve as input for the sentiment analysis models in the next stage.

First, we'll run test datafram through the N-grams generation, Bag of Words, TF-IDF.

In [ ]:
# load lemmatized_df if memory crashes
# path_to_saved_lemmatized_file = os.path.join(data_dir, "lemmatized_df.csv")  # Path to saved file
# lemmatized_df = pd.read_csv(path_to_saved_lemmatized_file, low_memory=False)

In [ ]:
# Apply N-gram computation to test lemmatized twitter df
ngram_matrix_0, vectorizer_0 = compute_ngrams(preprocessed_df_test, \
                                          text_column='lemmatized_text', ngram_range=(2, 3))

# Feature names (n-grams)
ngram_features_0 = vectorizer_0.get_feature_names_out()

print(f"Top 10 n-grams: {ngram_features_0[:10]}")
print(f"Shape of n-gram matrix: {ngram_matrix_0.shape}")

In [ ]:
#Create a TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # can be adjusted as needed

In [ ]:
# Fit the vectorizer to test lemmatized twitter df
tfidf_matrix = tfidf_vectorizer.fit_transform(preprocessed_df_test['lemmatized_text'])

## Visualize the results
# tfidf_matrix and tfidf_vectorizer
feature_names = tfidf_vectorizer.get_feature_names_out()
tfidf_scores = tfidf_matrix.toarray().sum(axis=0)

wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(dict(zip(feature_names, tfidf_scores)))

plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
# Bag of Words for the test matrix
bow_vectorizer = TfidfVectorizer(max_features=5000)  #  can be adjusted as needed

In [ ]:
# Fit the vectorizer to test lemmatized twitter df
bow_matrix = bow_vectorizer.fit_transform(preprocessed_df_test['lemmatized_text'])

# Plot heatmap
subset_matrix = bow_matrix.toarray()

plt.figure(figsize=(12, 10))
sns.heatmap(subset_matrix, cmap="viridis", annot=True, fmt=".2g",
            xticklabels=bow_vectorizer.get_feature_names_out(),
            yticklabels=range(bow_matrix.shape[0]))
plt.title("Bag of Words Heatmap (Test Data)")
plt.xlabel("Words")
plt.ylabel("Tweets")
plt.show()

Extracting N-grams for the entire datset

In [ ]:
ngram_matrix_twitter, vectorizer_twitter = compute_ngrams(lemmatized_df, \
                                          text_column='lemmatized_text', ngram_range=(2, 3))

# Feature names (n-grams)
ngram_features_twitter = vectorizer_twitter.get_feature_names_out()

print(f"Top 10 n-grams: {ngram_features_twitter[:10]}")
print(f"Shape of n-gram matrix: {ngram_features_twitter.shape}")

How N-grams are distributed by sentiments?

In [ ]:
# Convert the n-gram matrix to a DataFrame
ngram_df = pd.DataFrame(
    ngram_matrix_twitter.toarray(),
    columns=ngram_features_twitter
)
ngram_df['sentiment'] = lemmatized_df['sentiment']  # Add the sentiment column

# Aggregate n-grams by sentiment
sentiment_ngrams = ngram_df.groupby('sentiment').sum()

# Step 3: Select top n-grams for each sentiment
top_ngrams_per_sentiment = {}
n_top = 10
for sentiment in sentiment_ngrams.index:
    top_ngrams = sentiment_ngrams.loc[sentiment].nlargest(n_top)
    top_ngrams_per_sentiment[sentiment] = top_ngrams

# Step 4: Visualize the top n-grams for each sentiment
plt.figure(figsize=(16, 8))

for i, (sentiment, top_ngrams) in enumerate(top_ngrams_per_sentiment.items(), 1):
    plt.subplot(1, 3, i)
    sns.barplot(
    x=top_ngrams.values,
    y=top_ngrams.index,
    hue=top_ngrams.index,  # Assign hue to the y variable
    dodge=False,           # Ensure no splitting
    legend=False,          # Disable the legend
    palette='viridis'
    )
    plt.title(f"Top {n_top} N-grams ({sentiment})", fontsize=14)
    plt.xlabel('Frequency', fontsize=12)
    plt.ylabel('N-Gram', fontsize=12)

plt.tight_layout()
plt.show()

In [ ]:
# Fit the BoW vectorizer to full twitter data
bow_matrix = bow_vectorizer.fit_transform(lemmatized_df['lemmatized_text'])

In [ ]:
# Sum frequencies for each term
term_frequencies = bow_matrix.sum(axis=0).A1  # Sum along columns
top_indices = term_frequencies.argsort()[-20:][::-1]  # Indices of top 20 terms

# Subset matrix for these terms
subset_matrix = bow_matrix[:, top_indices][:20].toarray()

plt.figure(figsize=(10, 8))
sns.heatmap(subset_matrix, cmap='viridis', annot=True, fmt=".2f",
            xticklabels=bow_vectorizer.get_feature_names_out()[top_indices],  # Feature names for top terms
            yticklabels=range(20))
plt.title('Heatmap of Top 20 Terms (BoW)')
plt.show()

In [ ]:
# Fit the tf-idf vectorizer to full twitter data
tfidf_matrix = tfidf_vectorizer.fit_transform(lemmatized_df['lemmatized_text'])

In [ ]:
## Visualize the results
# Compute term frequencies
tfidf_scores = np.asarray(tfidf_matrix.sum(axis=0)).flatten()

# Get feature names from the tfidf_vectorizer
feature_names = tfidf_vectorizer.get_feature_names_out()

# Select top 200 terms (adjust if needed)
top_indices = np.argsort(tfidf_scores)[-200:]
top_features = feature_names[top_indices] # Now, feature_names is from tfidf_vectorizer
top_scores = tfidf_scores[top_indices]

wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(
    dict(zip(top_features, top_scores))
)

plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

Embeddings (BERT).

In [ ]:
# Load pre-trained BERT model and tokenizer
# bert_model_name = "bert-base-uncased"
# tokenizer = BertTokenizer.from_pretrained(bert_model_name)
# model = BertModel.from_pretrained(bert_model_name)

# Apply BERT embeddings to dataset with progress bar
# lemmatized_df['bert_embedding'] = compute_bert_embeddings_parallel(lemmatized_df, text_column='lemmatized_text')

In [ ]:
# save lemmatize_df
#lemmatized_df.to_csv("lemmatized_df_with_bert.csv", index=False)
#print("lemmatized_df has been saved as 'lemmatized_df_with_bert.csv'.")

In [ ]:
# Load GloVe embeddings
glove_file = "glove.twitter.27B.200d.txt"  # 200-dimensional embeddings
glove_path = os.path.join(data_dir, glove_file)
embeddings_dict = load_glove_embeddings(glove_path)

#Compute embeddings for a dataset
lemmatized_df['sentence_embedding'] = lemmatized_df['lemmatized_text'].apply(
    lambda x: compute_sentence_embedding(x, embeddings_dict, embedding_dim=200)
)

In [ ]:
# Getting features ready for use:
# Convert embeddings into a 2D NumPy array
embeddings_matrix = np.vstack(lemmatized_df['sentence_embedding'].values)
embeddings_sparse = csr_matrix(embeddings_matrix)

# 4. Sentiment Classification

This section aims to analyze the sentiment expressed in the preprocessed tweets using machine learning models. The goal is to classify tweets into different sentiment categories (e.g., positive, negative, neutral) and to understand the overall sentiment trends within the dataset. This involves:

* Model Selection and Training: Choosing a suitable classification model and training it on the preprocessed tweet data. We'll start with a baseline simple model.

* Model Evaluation: Assessing the performance of the trained model using appropriate metrics.
* Insights and Visualization: Presenting the results of the sentiment analysis, including visualizations and insights derived from the model's predictions.


**Input Overview**

* Our input features `X` will be a combination (concatenation) of computed text features:
`bow_matrix`, `tfidf_matrix`, `ngram_matrix_twitter`, `embeddings_sparse`.
* Our target variable `y` is the sentiment : `{neg, neu, pos}`.
* Our performance metric will be `f1-score`.   
* X dimentions: `170675x10200`.
* Sentiment distribution (class imbalance):  
`neu:76894, pos:50239, neg:43542`.


In [ ]:
# Features and labels
X_bow = hstack([bow_matrix, ngram_matrix_twitter, embeddings_sparse])
X_tfidf = hstack([tfidf_matrix, ngram_matrix_twitter, embeddings_sparse])
X_bow.shape

In [ ]:
# check sentiment distribution between all 3 versions:
print(lemmatized_df['sentiment'].value_counts())
print(lemmatized_df['new_simple_sentiment'].value_counts())
print(lemmatized_df['new_advanced_sentiment'].value_counts())

##  4.1 Train-Test Data Split.
 Split proportions: 70% training, 15% testing, 15% validation.

In [ ]:
y = lemmatized_df['sentiment']     # Sentiment labels
# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the sentiment labels
y_encoded = label_encoder.fit_transform(y)  # y contains 'pos', 'neg', 'neu'

# Check the mapping
print("Label Mapping:", dict(zip(label_encoder.classes_, range(len(label_encoder.classes_)))))

In [ ]:
# Split the data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X_bow, y_encoded, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [ ]:
# size of each set
print(f"Training set size: {X_train.shape[0]}")
print(f"Validation set size: {X_val.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")

# Number of features
print(f"Number of features: {X_train.shape[1]}")

In [ ]:
# save features and variables for future use
pd.to_pickle(X_train, os.path.join(data_dir, "X_train.pkl"))
pd.to_pickle(X_val, os.path.join(data_dir, "X_val.pkl"))
pd.to_pickle(X_test, os.path.join(data_dir, "X_test.pkl"))
pd.to_pickle(y_train, os.path.join(data_dir, "y_train.pkl"))
pd.to_pickle(y_val, os.path.join(data_dir, "y_val.pkl"))
pd.to_pickle(y_test, os.path.join(data_dir, "y_test.pkl"))
pd.to_pickle(label_encoder, os.path.join(data_dir, "label_encoder.pkl"))
pd.to_pickle(bow_vectorizer, os.path.join(data_dir, "bow_vectorizer.pkl"))
pd.to_pickle(ngram_features_twitter, os.path.join(data_dir, "vectorizer_twitter.pkl"))
pd.to_pickle(embeddings_dict, os.path.join(data_dir, "embeddings_dict.pkl"))

## 4.2 Sentiment Classification with Simple Logistic Regression Model.

In [ ]:
# Logistic Regression Model
log_reg = LogisticRegression(
    penalty='l2',  # L2 regularization (Ridge regression penalty)
    C=1.0,  # Regularization strength (smaller = stronger regularization)
    max_iter=1000,  # Ensure convergence
    random_state=42
)

# Train the model
log_reg.fit(X_train, y_train)

In [ ]:
# Predict
y_pred_log = log_reg.predict(X_test)
y_val_log = log_reg.predict(X_val)

# Decode labels for y_test and y_val

y_test_decoded = label_encoder.inverse_transform(y_test)
y_val_decoded = label_encoder.inverse_transform(y_val)
y_pred_log_decoded = label_encoder.inverse_transform(y_pred_log)
y_val_log_decoded = label_encoder.inverse_transform(y_val_log)

# Compute classification report for test set
print("Base Model LogReg Classification Report for Test Set:")
print(classification_report(y_test_decoded, y_pred_log_decoded, target_names=label_encoder.classes_))

# Compute classification report for validation set
print("Base Model LogReg Classification Report for Validation Set:")
print(classification_report(y_val_decoded, y_val_log_decoded, target_names=label_encoder.classes_))

In [ ]:
# Compute confusion matrix
cm = confusion_matrix(y_test_decoded, y_pred_log_decoded, labels=label_encoder.classes_)

# Normalize confusion matrix by rows to show percentages
cm_percent = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100  # Convert to percentages

# Display the normalized confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm_percent, display_labels=label_encoder.classes_)
fig, ax = plt.subplots(figsize=(8, 6))
disp.plot(cmap='Blues', colorbar=True, ax=ax)

plt.title("LogReg Confusion Matrix (Percentages) on Test Set")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

## 4.3 Tuning Hyperparameters for Base Logisstic Regression Model

In [ ]:
# Define parameter grid
param_grid = {
    'penalty': ['l1', 'l2'],
    'C': [0.01, 0.1, 1, 10, 100],
    'solver': ['liblinear', 'saga'],
    'class_weight': [None, 'balanced']
}

# Create Logistic Regression model
log_reg = LogisticRegression(max_iter=1000, random_state=42)

# Grid search with F1 scoring
grid_search = GridSearchCV(
    estimator=log_reg,
    param_grid=param_grid,
    scoring=make_scorer(f1_score, average='weighted'),  # Weighted F1 score
    cv=5,
    verbose=2,
    n_jobs=-1
)

# Fit on the training data
grid_search.fit(X_train, y_train)

# Best parameters and F1 score
print("Best Parameters:", grid_search.best_params_)
print("Best F1 Score:", grid_search.best_score_)

## 4.4 Improved Model Performance and Feature Interpretation.

## 4.5 Feature Reduction (Experimental)

In [ ]:
# Ensure X_bow is dense
X_dense = csr_matrix(X_bow).toarray()

# Use the hybrid GWO + ABC optimizer
best_features = hybrid_gwo_abc(X_dense, y_encoded, model=SVC(kernel='linear'))

In [ ]:
# Apply the best feature mask
selected_indices = np.where(best_features == 1)[0]
X_bow_selected = X_bow[:, selected_indices]

print(f"Reduced feature set from {X_bow.shape[1]} to {X_bow_selected.shape[1]}")

In [ ]:
# Train-test-val split
X_train, X_temp, y_train, y_temp = train_test_split(X_bow_selected, y_encoded, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
# Train SVM
model = SVC(kernel='linear')
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.2f}")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

## 4.5 Other Tested Classification Models.

## 4.5.1 SDG Classifier Performance.

In [ ]:
# SGDClassifier with logistic regression loss
sgd_model = SGDClassifier(
    loss='log_loss',          # Logistic regression loss
    penalty='l2',             # L2 regularization
    alpha=1e-4,               # Regularization strength
    max_iter=1000,            # Maximum number of iterations
    tol=1e-3,                 # Convergence tolerance
    random_state=42
)

# Train and evaluate the SGD model
sgd_model.fit(X_train, y_train)
y_pred_sgd = sgd_model.predict(X_test)
y_val_sgd = sgd_model.predict(X_val)

# Decode labels for y_test and y_pred_sgd
y_test_decoded = label_encoder.inverse_transform(y_test)
y_val_decoded = label_encoder.inverse_transform(y_val)
y_pred_sgd_decoded = label_encoder.inverse_transform(y_pred_sgd)
y_val_sgd_decoded = label_encoder.inverse_transform(y_val_sgd)

# Compute classification report for test set
print("Classification Report for Test Set:")
print(classification_report(y_test_decoded, y_pred_sgd_decoded, target_names=label_encoder.classes_))

# Compute classification report for validation set
print("Classification Report for Validation Set:")
print(classification_report(y_val_decoded, y_val_sgd_decoded, target_names=label_encoder.classes_))

In [ ]:
coefficients = pipeline_sgd.named_steps['sgd'].coef_

# Get feature names and align them with the combined feature matrix
bow_features = bow_vectorizer.get_feature_names_out()
ngram_features = vectorizer_twitter.get_feature_names_out()
embedding_dim = 200
embedding_feature_names = [f"embedding_{i}" for i in range(embedding_dim)]

all_features = np.concatenate([bow_features, ngram_features, embedding_feature_names])  # Combine feature names

# Ensure alignment between coefficients and feature names
if coefficients.shape[1] != len(all_features):
    raise ValueError("Mismatch between coefficients and feature names.")

# Analyze top features for each sentiment
for i, sentiment in enumerate(label_encoder.classes_):
    top_indices = np.argsort(coefficients[i])[-10:]  # Top 10 features for each sentiment
    top_features = all_features[top_indices]  # Use combined features

    print(f"\nTop Features for {sentiment.capitalize()} Sentiment:")
    for feature, coef in zip(top_features, coefficients[i][top_indices]):
        if feature.startswith("embedding_"):
            # Extract the embedding index from the feature name (e.g., embedding_11 -> 11)
            embedding_index = int(feature.split("_")[1])
            print(f"{feature} (Embedding Value): {coef:.4f}")
        else:
            print(f"{feature}: {coef:.4f}")

## 4.5.2. Random Forest

In [ ]:
# Random Forest model with specific parameters
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=20,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42
)

rf_model.fit(X_train, y_train)
# Train and evaluate the RF model

y_pred_rf= rf_model.predict(X_test)
y_val_rf = rf_model.predict(X_val)

# Decode labels for y_test and y_pred_rf
y_test_decoded = label_encoder.inverse_transform(y_test)
y_val_decoded = label_encoder.inverse_transform(y_val)
y_pred_rf_decoded = label_encoder.inverse_transform(y_pred_rf)
y_val_rf_decoded = label_encoder.inverse_transform(y_val_rf)

# Compute classification report for test set
print("Random Forest Classification Report for Test Set:")
print(classification_report(y_test_decoded, y_pred_rf_decoded, target_names=label_encoder.classes_))

# Compute classification report for validation set
print("Random Forest Classification Report for Validation Set:")
print(classification_report(y_val_decoded, y_val_rf_decoded, target_names=label_encoder.classes_))

In [ ]:
# Desicion Tree with specific parameters
dt_model =DecisionTreeClassifier(criterion = 'gini',
                                 max_depth = 20,
                                 min_samples_leaf = 2,
                                 min_samples_split = 10,
                                 random_state=42)


# Train and evaluate the DT model
dt_model.fit(X_train, y_train)
y_pred_dt= dt_model.predict(X_test)
y_val_dt = dt_model.predict(X_val)

# Decode labels for y_test and y_pred_dt
y_test_decoded = label_encoder.inverse_transform(y_test)
y_val_decoded = label_encoder.inverse_transform(y_val)
y_pred_dt_decoded = label_encoder.inverse_transform(y_pred_dt)
y_val_dt_decoded = label_encoder.inverse_transform(y_val_dt)

# Compute classification report for test set
print("Decision Trees Classification Report for Test Set:")
print(classification_report(y_test_decoded, y_pred_dt_decoded, target_names=label_encoder.classes_))

# Compute classification report for validation set
print("Decision Trees Classification Report for Validation Set:")
print(classification_report(y_val_decoded, y_val_dt_decoded, target_names=label_encoder.classes_))

## 4.5.3 XGBoost

In [ ]:
# Define the XGBoost model with specific parameters
xgb_model = XGBClassifier(learning_rate=0.2, max_depth=10, n_estimators=200)

# Train and evaluate the XGBoost model
xgb_model.fit(X_train, y_train)

y_pred_xgb= xgb_model.predict(X_test)
y_val_xgb = xgb_model.predict(X_val)

# Decode labels for y_test and y_pred_xgb
y_test_decoded = label_encoder.inverse_transform(y_test)
y_val_decoded = label_encoder.inverse_transform(y_val)
y_pred_xgb_decoded = label_encoder.inverse_transform(y_pred_xgb)
y_val_xgb_decoded = label_encoder.inverse_transform(y_val_xgb)

# Compute classification report for test set
print("XGB Classification Report for Test Set:")
print(classification_report(y_test_decoded, y_pred_xgb_decoded, target_names=label_encoder.classes_))

# Compute classification report for validation set
print("XGB Classification Report for Validation Set:")
print(classification_report(y_val_decoded, y_val_xgb_decoded, target_names=label_encoder.classes_))

# 5. Conclusion.